WORKFLOW:

- for every row in statcast 
- grab game_pk 
- dump into statsapi.schedule 
- grab starttime, venue 
- translate venue into lat, long with coord.csv 
- dump into visual crossing 
- grab weather info 
- dump starttime, venue, and weather info back into statcast df 
- grab pitcher id from df 
- run through pybaseball.statcast_pitcher to grab "relevent" stats through DAY BEFORE GAME 
- dump stats in df 
- grab batter id from df 
- run through pybaseball.statcast_batter to grab "relevent" stata through DAY BEFORE GAME 
- dump stats in df

IMPLEMENT CACHING - have relevent parts of code check their caches first:

save game_pk results and their matching starttime, location, weather - no need to ever clear

save pitcher and batter stats - clear after every day (not game because you can't wager between double headers)


according to retrosheets the day/night cutoff is 5PM

Import data from statcast

In [209]:
from pybaseball import statcast, statcast_pitcher, statcast_batter, playerid_lookup
import pandas as pd
pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', 10)
import numpy as np
import statsapi

In [ ]:
data = statcast('2019-03-20', '2019-09-29')

data.to_csv('2019_statcast_pbp.csv', index=False)

data = statcast('2020-07-23', '2020-09-27')

data.to_csv('2020_statcast_pbp.csv', index=False)

data = statcast('2021-04-01', '2021-06-30')

data.to_csv('2021_statcast_pbp.csv', index=False)

In [2]:
df = pd.concat([pd.read_csv('2019_statcast_pbp.csv'),
                pd.read_csv('2020_statcast_pbp.csv'),
               pd.read_csv('2021_statcast_pbp.csv')], ignore_index=True)

In [3]:
df['bottom'] = df['inning_topbot'].apply(lambda x: 1 if x == 'Bot' else 0)

In [4]:
df = df.drop(labels=['inning_topbot'], axis=1)

In [5]:
df['game_date'] = pd.to_datetime(df['game_date'])
df['game_date'] = df['game_date'].apply(lambda x: x.toordinal())

In [6]:
df = df.sort_values(['game_date', 'game_pk','home_team', 'inning', 'bottom', 'outs_when_up', 'at_bat_number', 'pitch_number']).reset_index(drop=True)

In [ ]:
df.head(20)

In [ ]:
df.game_type.value_counts()

In [ ]:
df.description.value_counts()

In [ ]:
df.events.value_counts()

In [ ]:
df.head()

In [ ]:
from pybaseball import playerid_reverse_lookup

playerid_reverse_lookup([547982, 543208], key_type='mlbam')

In [ ]:
df[df['events'] == 'other_out']

In [ ]:
# class PlateAppearence:
#     def __init__(self, row):
#         self.balls = row['balls']
#         self.strikes = row['strikes']
#         self.batter = row['batter']
#         self.pitcher = row['pitcher']
#         self.inning = row['inning']
#         self.bottom = row['bottom']
#         self.outs = row['outs_when_up']
        
    

In [ ]:
# for index, row in df.iterrows():
#     pa = PlateAppearence(row)
    
#     try:
#         pa == 
#     prev_row = row

In [7]:
pa_ending_events = ['field_out',
                    'strikeout',
                    'single',
                    'walk',
                    'double',
                    'home_run',
                    'force_out',
                    'grounded_into_double_play',
                    'hit_by_pitch',
                    'field_error',
                    'sac_fly',
                    'triple',
                    'sac_bunt',
                    'fielders_choice',
                    'double_play',
                    'fielders_choice_out',
                    'strikeout_double_play',
                    'catcher_interf',
                    'sac_fly_double_play',
                    'triple_play',
                    'sac_bunt_double_play']

df_filtered = df[df['events'].isin(pa_ending_events)]

In [276]:
hit_events = ['single',
            'double',
            'home_run',
            'triple']

In [ ]:
df_filtered

Sanity test to make sure rows are in expected order

In [8]:
previous_batter = {'batter': 0}

for index, row in df_filtered.iterrows():
    if previous_batter['batter'] != row['batter']:
        previous_batter = row
        previous_index = index
        pass
    elif previous_batter['inning'] == 9 and previous_batter['outs_when_up'] == 2 and row['at_bat_number'] == 1:
            previous_batter = row
            previous_index = index
            pass
    else:
        print(previous_index, previous_batter)
        print(index, row)
        break

load in Parks.csv as pd

In [36]:
venue_coords = pd.read_csv('Parks.csv')

In [ ]:
venue_coords

In [37]:
game_pk_df = pd.DataFrame(columns = statsapi.schedule(game_id=566083)[0].keys())

In [38]:
# venue_dict = {}

for index, row in df_filtered.iterrows():
    game_pk = row['game_pk']
    if game_pk in game_pk_df['game_id'].unique():
        starttime = game_pk_df[game_pk_df['game_id'] == game_pk]['game_datetime'].iloc[0]
        venue = game_pk_df[game_pk_df['game_id'] == game_pk]['venue_name'].iloc[0]
    else:
        game_data = statsapi.schedule(game_id = game_pk)[0]
        starttime = game_data['game_datetime']
        venue = game_data['venue_name']
        game_pk_df = game_pk_df.append(game_data, ignore_index=True)
        print('***************', venue)
#     if venue in ['Tokyo Dome',
#                  'Estadio de Beisbol Monterrey',
#                  'TD Ameritrade Park',
#                  'London Stadium',
#                  'BB&T Ballpark']:
#         pass
#     else:
    try:
        city = venue_coords[venue_coords['NAME'] == venue]['CITY'].iloc[0]
        lat = venue_coords[venue_coords['NAME'] == venue]['Latitude'].iloc[0]
        lon = venue_coords[venue_coords['NAME'] == venue]['Longitude'].iloc[0]
        alt = venue_coords[venue_coords['NAME'] == venue]['Altitude'].iloc[0]
        venue_dict[venue] = np.array((city, lat, lon, alt))
    except IndexError:
        raise IndexError(venue, index)
#     print(venue_dict)

*************** Tokyo Dome
*************** Tokyo Dome
*************** Great American Ball Park
*************** Kauffman Stadium
*************** Dodger Stadium
*************** Nationals Park
*************** Oakland Coliseum
*************** Petco Park
*************** T-Mobile Park
*************** Tropicana Field
*************** Globe Life Park in Arlington
*************** Rogers Centre
*************** Target Field
*************** Citizens Bank Park
*************** Marlins Park
*************** Yankee Stadium
*************** Miller Park
*************** Dodger Stadium
*************** Oakland Coliseum
*************** Petco Park
*************** T-Mobile Park
*************** Tropicana Field
*************** Rogers Centre
*************** Marlins Park
*************** Miller Park
*************** Kauffman Stadium
*************** Dodger Stadium
*************** Nationals Park
*************** Oakland Coliseum
*************** Petco Park
*************** T-Mobile Park
*************** Tropicana Field
****

*************** Nationals Park
*************** Oakland Coliseum
*************** T-Mobile Park
*************** Tropicana Field
*************** Globe Life Park in Arlington
*************** Target Field
*************** Citizens Bank Park
*************** SunTrust Park
*************** Guaranteed Rate Field
*************** Marlins Park
*************** Yankee Stadium
*************** Miller Park
*************** Angel Stadium
*************** Coors Field
*************** Comerica Park
*************** Nationals Park
*************** Petco Park
*************** Tropicana Field
*************** Target Field
*************** SunTrust Park
*************** Yankee Stadium
*************** Miller Park
*************** Angel Stadium
*************** Wrigley Field
*************** Coors Field
*************** Comerica Park
*************** Oakland Coliseum
*************** PNC Park
*************** Petco Park
*************** Busch Stadium
*************** Tropicana Field
*************** Globe Life Park in Arlington
***

*************** Oriole Park at Camden Yards
*************** Wrigley Field
*************** Progressive Field
*************** Coors Field
*************** Comerica Park
*************** Minute Maid Park
*************** Dodger Stadium
*************** Oakland Coliseum
*************** PNC Park
*************** Petco Park
*************** Busch Stadium
*************** Tropicana Field
*************** Rogers Centre
*************** Yankee Stadium
*************** Miller Park
*************** Oriole Park at Camden Yards
*************** Wrigley Field
*************** Progressive Field
*************** Comerica Park
*************** Minute Maid Park
*************** Dodger Stadium
*************** Oakland Coliseum
*************** PNC Park
*************** Petco Park
*************** Busch Stadium
*************** Tropicana Field
*************** Rogers Centre
*************** Yankee Stadium
*************** Miller Park
*************** Chase Field
*************** Wrigley Field
*************** Progressive Field
****

*************** PNC Park
*************** Oracle Park
*************** Busch Stadium
*************** Rogers Centre
*************** Target Field
*************** Miller Park
*************** Oriole Park at Camden Yards
*************** Fenway Park
*************** Great American Ball Park
*************** Great American Ball Park
*************** Coors Field
*************** Minute Maid Park
*************** Dodger Stadium
*************** Nationals Park
*************** Oakland Coliseum
*************** T-Mobile Park
*************** Tropicana Field
*************** Target Field
*************** Guaranteed Rate Field
*************** Yankee Stadium
*************** Oriole Park at Camden Yards
*************** Fenway Park
*************** Great American Ball Park
*************** Coors Field
*************** Minute Maid Park
*************** Dodger Stadium
*************** Oakland Coliseum
*************** T-Mobile Park
*************** Tropicana Field
*************** Target Field
*************** Citizens Bank P

*************** Marlins Park
*************** Oriole Park at Camden Yards
*************** Great American Ball Park
*************** Coors Field
*************** Comerica Park
*************** Minute Maid Park
*************** Dodger Stadium
*************** Nationals Park
*************** Citi Field
*************** Oakland Coliseum
*************** Oracle Park
*************** Tropicana Field
*************** Target Field
*************** SunTrust Park
*************** Guaranteed Rate Field
*************** Marlins Park
*************** Oriole Park at Camden Yards
*************** Great American Ball Park
*************** Coors Field
*************** Comerica Park
*************** Minute Maid Park
*************** Dodger Stadium
*************** Nationals Park
*************** Citi Field
*************** Oakland Coliseum
*************** Oracle Park
*************** Tropicana Field
*************** Target Field
*************** SunTrust Park
*************** Guaranteed Rate Field
*************** Marlins Park
***

...

*************** Wrigley Field
*************** Angel Stadium
*************** Fenway Park
*************** loanDepot park
*************** Yankee Stadium
*************** Comerica Park
*************** Citizens Bank Park
*************** T-Mobile Park
*************** Coors Field
*************** Nationals Park
*************** Oakland Coliseum
*************** Globe Life Field
*************** Nationals Park
*************** Yankee Stadium
*************** Coors Field
*************** Wrigley Field
*************** Petco Park
*************** Oakland Coliseum
*************** Great American Ball Park
*************** Globe Life Field
*************** loanDepot park
*************** Progressive Field
*************** T-Mobile Park
*************** Nationals Park
*************** Comerica Park
*************** Citizens Bank Park
*************** Fenway Park
*************** Oriole Park at Camden Yards
*************** TD Ballpark
*************** Minute Maid Park
*************** Target Field
*************** PNC Par

*************** Minute Maid Park
*************** Progressive Field
*************** Truist Park
*************** Oriole Park at Camden Yards
*************** Comerica Park
*************** American Family Field
*************** Busch Stadium
*************** American Family Field
*************** PNC Park
*************** Guaranteed Rate Field
*************** Minute Maid Park
*************** TD Ballpark
*************** Progressive Field
*************** Truist Park
*************** Oracle Park
*************** Globe Life Field
*************** Chase Field
*************** Tropicana Field
*************** Dodger Stadium
*************** Oriole Park at Camden Yards
*************** Citi Field
*************** Tropicana Field
*************** Oracle Park
*************** Citi Field
*************** Busch Stadium
*************** PNC Park
*************** American Family Field
*************** Chase Field
*************** TD Ballpark
*************** Globe Life Field
*************** Minute Maid Park
**************

*************** Minute Maid Park
*************** Minute Maid Park
*************** TD Ballpark
*************** Tropicana Field
*************** Comerica Park
*************** Oriole Park at Camden Yards
*************** T-Mobile Park
*************** Petco Park
*************** PNC Park
*************** Coors Field
*************** Fenway Park
*************** Dodger Stadium
*************** Guaranteed Rate Field
*************** Chase Field
*************** American Family Field
*************** Target Field
*************** Petco Park
*************** Truist Park
*************** Great American Ball Park
*************** Wrigley Field
*************** Target Field
*************** T-Mobile Park
*************** Dodger Stadium
*************** Globe Life Field
*************** Angel Stadium
*************** Kauffman Stadium
*************** Oakland Coliseum
*************** Citizens Bank Park
*************** Wrigley Field
*************** Great American Ball Park
*************** Petco Park
*************** Glob

*************** Globe Life Field
*************** Angel Stadium
*************** Kauffman Stadium
*************** Guaranteed Rate Field
*************** American Family Field
*************** PNC Park
*************** Citizens Bank Park
*************** Coors Field
*************** PNC Park
*************** Citizens Bank Park
*************** Busch Stadium
*************** Kauffman Stadium
*************** Sahlen Field
*************** Oriole Park at Camden Yards
*************** Petco Park
*************** American Family Field
*************** Truist Park
*************** Globe Life Field
*************** Angel Stadium
*************** Coors Field
*************** Yankee Stadium
*************** Oracle Park
*************** Guaranteed Rate Field
*************** Guaranteed Rate Field
*************** Sahlen Field
*************** Citizens Bank Park
*************** PNC Park
*************** American Family Field
*************** Globe Life Field
*************** Petco Park
*************** Busch Stadium
********

*************** Dodger Stadium
*************** Great American Ball Park
*************** Tropicana Field
*************** Target Field
*************** loanDepot park
*************** Yankee Stadium
*************** Sahlen Field
*************** Comerica Park
*************** Petco Park
*************** Busch Stadium
*************** Great American Ball Park
*************** Oracle Park
*************** Tropicana Field
*************** Sahlen Field
*************** Fenway Park
*************** Citi Field
*************** Target Field
*************** Globe Life Field
*************** loanDepot park
*************** American Family Field
*************** Dodger Stadium
*************** Guaranteed Rate Field
*************** Citi Field
*************** Oracle Park
*************** Guaranteed Rate Field
*************** Comerica Park
*************** Great American Ball Park
*************** Petco Park
*************** American Family Field
*************** Sahlen Field
*************** Tropicana Field
**************

In [111]:
df_filtered.tail()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bottom
1345219,FF,737971,95.4,-0.32,6.44,"Karinchak, James",606992,675916,field_out,hit_into_play,NaN,NaN,NaN,NaN,14.0,Eric Haase flies out to right fielder Harold R...,R,R,R,CLE,DET,X,9.0,fly_ball,0,1,2021,-0.03,1.11,0.93,2.39,NaN,NaN,663662.0,1,7,197.69,116.56,NaN,NaN,425784.0,NaN,NaN,3.363890,-138.801269,-7.549467,-1.108641,29.290681,-16.421232,3.30,1.50,270.0,80.0,39.0,95.5,2300.0,6.2,633506,675916,425784.0,656252.0,676391.0,644374.0,642708.0,592696.0,640458.0,623912.0,54.26,0.030,0.036,0.0,1.0,0.0,0.0,3.0,57,2,4-Seam Fastball,1,7,7,1,7,1,7,1,Standard,Standard,193.0,0.000,-0.261,0
1345223,CU,737971,83.3,-0.18,6.45,"Karinchak, James",670097,675916,strikeout,swinging_strike_blocked,NaN,NaN,NaN,NaN,14.0,"Zack Short strikes out swinging, catcher Rene ...",R,R,R,CLE,DET,S,2.0,NaN,1,2,2021,-0.12,-0.80,0.48,0.60,NaN,NaN,663662.0,2,7,NaN,NaN,NaN,NaN,425784.0,NaN,NaN,1.754642,-121.293004,-5.424961,-1.478941,22.285520,-39.355920,3.30,1.50,NaN,NaN,NaN,83.2,2245.0,6.0,633506,675916,425784.0,656252.0,676391.0,644374.0,642708.0,592696.0,640458.0,623912.0,54.48,NaN,NaN,0.0,1.0,0.0,0.0,NaN,58,4,Curveball,1,7,7,1,7,1,7,1,Standard,Standard,18.0,0.001,-0.150,0
1345227,FF,737971,95.4,-2.25,6.05,"Jiménez, Joe",640458,641729,strikeout,called_strike,NaN,NaN,NaN,NaN,1.0,Oscar Mercado called out on strikes.,R,R,R,CLE,DET,S,2.0,NaN,1,2,2021,-0.52,1.48,-0.42,3.20,NaN,NaN,NaN,0,7,NaN,NaN,NaN,NaN,606992.0,NaN,NaN,5.974446,-138.764180,-5.198531,-7.971755,30.132469,-12.013564,3.32,1.63,NaN,NaN,NaN,95.6,2387.0,6.4,633506,641729,606992.0,570731.0,650489.0,600869.0,670097.0,543257.0,656537.0,663662.0,54.15,NaN,NaN,0.0,1.0,0.0,0.0,NaN,59,4,4-Seam Fastball,1,7,1,7,7,1,1,7,Standard,Standard,213.0,-0.002,-0.172,1
1345232,SL,737971,84.5,-2.14,6.06,"Jiménez, Joe",680911,641729,strikeout,swinging_strike,NaN,NaN,NaN,NaN,9.0,Owen Miller strikes out swinging.,R,R,R,CLE,DET,S,2.0,NaN,0,2,2021,0.07,0.24,0.66,1.69,NaN,NaN,NaN,1,7,NaN,NaN,NaN,NaN,606992.0,NaN,NaN,6.388622,-122.984828,-4.317439,-0.579978,24.169849,-29.061171,3.37,1.53,NaN,NaN,NaN,84.6,2078.0,6.3,633506,641729,606992.0,570731.0,650489.0,600869.0,670097.0,543257.0,656537.0,663662.0,54.16,NaN,NaN,0.0,1.0,0.0,0.0,NaN,60,5,Slider,1,7,1,7,7,1,1,7,Standard,Standard,135.0,-0.001,-0.099,1
1345235,FF,737971,94.5,-2.19,5.89,"Jiménez, Joe",644374,641729,field_out,hit_into_play,NaN,NaN,NaN,NaN,3.0,Yu Chang pops out to second baseman Willi Castro.,R,R,R,CLE,DET,X,4.0,popup,2,0,2021,-0.56,1.28,0.59,2.94,NaN,NaN,NaN,2,7,151.05,124.07,NaN,NaN,606992.0,NaN,NaN,8.555852,-137.368361,-4.923537,-8.885372,26.884437,-14.968541,3.41,1.56,200.0,93.6,61.0,95.4,2283.0,6.6,633506,641729,606992.0,570731.0,650489.0,600869.0,670097.0,543257.0,656537.0,663662.0,53.90,0.007,0.007,0.0,1.0,0.0,0.0,3.0,61,3,4-Seam Fastball,1,7,1,7,7,1,1,7,Standard,Standard,208.0,0.000,-0.144,1


In [118]:
pd.DataFrame(venue_dict).to_csv('venue_dict')
df_filtered.to_csv('df_filtered')
game_pk_df.to_csv('game_pk_df')

In [5]:
venue_dict = pd.read_csv('venue_dict').drop('Unnamed: 0', axis = 1).to_dict()
df_filtered = pd.read_csv('df_filtered')
game_pk_df = pd.read_csv('game_pk_df')

weather work

In [27]:
game_pk_df

,Unnamed: 0,game_id,game_datetime,game_date,game_type,status,away_name,home_name,away_id,home_id,doubleheader,game_num,home_probable_pitcher,away_probable_pitcher,home_pitcher_note,away_pitcher_note,away_score,home_score,current_inning,inning_state,venue_id,venue_name,winning_team,losing_team,winning_pitcher,losing_pitcher,save_pitcher,summary
0,0,566083,2019-03-20T09:35:00Z,2019-03-20,R,Final,Seattle Mariners,Oakland Athletics,136,133,N,1,Mike Fiers,Marco Gonzales,"Fiers, who will be making the first Opening Da...",The 27-year-old gets his first Opening Day sta...,9,7,9.0,Bottom,2397,Tokyo Dome,Seattle Mariners,Oakland Athletics,Marco Gonzales,Mike Fiers,Hunter Strickland,2019-03-20 - Seattle Mariners (9) @ Oakland At...
1,1,566084,2019-03-21T09:35:00Z,2019-03-21,R,Final,Seattle Mariners,Oakland Athletics,136,133,N,1,Marco Estrada,Yusei Kikuchi,Estrada is seeking a bounce-back campaign as h...,After signing with the Mariners as a free agen...,5,4,12.0,Bottom,2397,Tokyo Dome,Seattle Mariners,Oakland Athletics,Zac Rosscup,Ryan Buchter,Hunter Strickland,2019-03-21 - Seattle Mariners (5) @ Oakland At...
2,2,565220,2019-03-28T20:10:00Z,2019-03-28,R,Final,Pittsburgh Pirates,Cincinnati Reds,134,113,N,1,Luis Castillo,Jameson Taillon,Their fifth different Opening Day starter in f...,Taillon will make his first Opening Day start ...,3,5,9.0,Top,2602,Great American Ball Park,Cincinnati Reds,Pittsburgh Pirates,Zach Duke,Jameson Taillon,David Hernandez,2019-03-28 - Pittsburgh Pirates (3) @ Cincinna...
3,3,565702,2019-03-28T20:15:00Z,2019-03-28,R,Final,Chicago White Sox,Kansas City Royals,145,118,N,1,Brad Keller,Carlos Rodon,Keller has gone from Rule 5 pick in 2017 to ma...,Rodon makes career Opening Day start No. 1 in ...,3,5,9.0,Top,7,Kauffman Stadium,Kansas City Royals,Chicago White Sox,Brad Keller,Carlos Rodon,Brad Boxberger,2019-03-28 - Chicago White Sox (3) @ Kansas Ci...
4,4,565799,2019-03-28T20:10:00Z,2019-03-28,R,Final,Arizona Diamondbacks,Los Angeles Dodgers,109,119,N,1,Hyun Jin Ryu,Zack Greinke,Injuries to Clayton Kershaw and Rich Hill leav...,Greinke will be making his third Opening Day s...,5,12,9.0,Top,22,Dodger Stadium,Los Angeles Dodgers,Arizona Diamondbacks,Hyun Jin Ryu,Zack Greinke,NaN,2019-03-28 - Arizona Diamondbacks (5) @ Los An...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4519,4519,633479,2021-07-01T00:10:00Z,2021-06-30,R,Final,Minnesota Twins,Chicago White Sox,142,145,N,1,Dylan Cease,Bailey Ober,NaN,NaN,3,13,9.0,Top,4,Guaranteed Rate Field,Chicago White Sox,Minnesota Twins,Dylan Cease,Bailey Ober,NaN,2021-06-30 - Minnesota Twins (3) @ Chicago Whi...
4520,4520,633484,2021-06-30T23:05:00Z,2021-06-30,R,Final,Los Angeles Angels,New York Yankees,108,147,N,1,Domingo German,Shohei Ohtani,NaN,NaN,11,8,9.0,Bottom,3313,Yankee Stadium,Los Angeles Angels,New York Yankees,Mike Mayers,Lucas Luetge,Raisel Iglesias,2021-06-30 - Los Angeles Angels (11) @ New Yor...
4521,4521,633485,2021-07-01T00:10:00Z,2021-06-30,R,Final,Baltimore Orioles,Houston Astros,110,117,N,1,Luis Garcia,Matt Harvey,NaN,NaN,5,2,9.0,Bottom,2392,Minute Maid Park,Baltimore Orioles,Houston Astros,Tanner Scott,Luis Garcia,Cole Sulser,2021-06-30 - Baltimore Orioles (5) @ Houston A...
4522,4522,633491,2021-06-30T19:10:00Z,2021-06-30,R,Final,Pittsburgh Pirates,Colorado Rockies,134,115,N,1,Jon Gray,Chad Kuhl,NaN,NaN,2,6,9.0,Top,19,Coors Field,Colorado Rockies,Pittsburgh Pirates,Jon Gray,Chad Kuhl,NaN,2021-06-30 - Pittsburgh Pirates (2) @ Colorado...


In [39]:
game_pk_df = game_pk_df.sort_values(['game_datetime'])

In [40]:
len(game_pk_df)

4524

0.0
0.00022104332449160034
0.0033156498673740055
0.0011052166224580018
0.0035366931918656055
0.0028735632183908046
0.002431476569407604
0.001989389920424403
0.0022104332449160036
0.001326259946949602
0.0015473032714412025
0.002652519893899204
0.0008841732979664014
0.003094606542882405
0.0004420866489832007
0.000663129973474801
0.0017683465959328027
0.004862953138815208
0.005083996463306808
0.004641909814323607
0.005305039787798408
0.003978779840848806
0.004420866489832007
0.004199823165340406
0.003757736516357206
0.008178603006189214
0.005968169761273209
0.007515473032714412
0.17374005305039789
0.005526083112290009
0.007294429708222812
0.007736516357206013
0.007957559681697613
0.006852343059239611
0.008399646330680813
0.007073386383731211
0.00641025641025641
0.00618921308576481
0.006631299734748011
0.005747126436781609
0.011494252873563218
0.010610079575596816
0.011273209549071617
0.008620689655172414
0.010167992926613616
0.009283819628647215
0.010831122900088417
0.011715296198054819
0

0.09217506631299735
0.09173297966401414
0.09239610963748895
0.09305923961096375
0.09283819628647215
0.09350132625994695
0.09328028293545534
0.09195402298850575
0.09261715296198055
0.09571175950486295
0.09438549955791335
0.09615384615384616
0.09416445623342175
0.09460654288240496
0.09593280282935455
0.09549071618037135
0.09637488947833775
0.09526967285587976
0.09946949602122016
0.09725906277630415
0.09394341290893014
0.09504862953138815
0.09372236958443855
0.09482758620689655
0.09703801945181255
0.09748010610079576
0.09858532272325375
0.09681697612732096
0.09969053934571176
0.09814323607427056
0.09902740937223696
0.09770114942528736
0.09991158267020336
0.09792219274977895
0.09924845269672856
0.10013262599469495
0.09880636604774536
0.09836427939876216
0.09659593280282935
0.10145888594164457
0.10101679929266136
0.10123784261715296
0.10190097259062776
0.10057471264367816
0.10079575596816977
0.10167992926613616
0.10035366931918656
0.10234305923961097
0.10477453580901856
0.10411140583554376


0.259946949602122
0.2595048629531388
0.2597259062776304
0.2619363395225464
0.26105216622458
0.2603890362511052
0.2608311229000884
0.25906277630415564
0.26061007957559684
0.258841732979664
0.2650309460654288
0.26282051282051283
0.26326259946949604
0.2625994694960212
0.2623784261715296
0.2639257294429708
0.2645888594164456
0.2634836427939876
0.26480990274093724
0.2641467727674624
0.2630415561450044
0.26215738284703805
0.26370468611847925
0.26436781609195403
0.26525198938992045
0.26702033598585323
0.26679929266136165
0.2661361626878868
0.26635720601237844
0.26746242263483644
0.2656940760389036
0.2672413793103448
0.26591511936339524
0.26657824933687
0.265473032714412
0.26790450928381965
0.26989389920424406
0.2703359858532272
0.2681255526083112
0.26856763925729443
0.26900972590627764
0.2696728558797524
0.27055702917771884
0.268788682581786
0.27011494252873564
0.26834659593280286
0.267683465959328
0.2692307692307692
0.26945181255526085
0.27122015915119363
0.27099911582670205
0.27320954907161

0.39986737400530503
0.39721485411140584
0.40008841732979666
0.3994252873563218
0.3974358974358974
0.39832007073386383
0.39854111405835546
0.39809902740937225
0.3989832007073386
0.39765694076038904
0.39964633068081346
0.39920424403183025
0.39699381078691426
0.3978779840848806
0.400972590627763
0.402077807250221
0.40163572060123787
0.40229885057471265
0.40075154730327145
0.40185676392572944
0.40141467727674623
0.4005305039787798
0.40119363395225466
0.40030946065428824
0.40561450044208663
0.40384615384615385
0.4051724137931034
0.40428824049513706
0.40495137046861185
0.40296198054818744
0.40318302387267907
0.40450928381962864
0.40539345711759506
0.40473032714412027
0.4036251105216622
0.40340406719717065
0.40274093722369586
0.40406719717064543
0.40251989389920423
0.40893015030946067
0.40760389036251105
0.40826702033598583
0.40738284703801947
0.4078249336870027
0.40649867374005305
0.40716180371352784
0.40848806366047746
0.4062776304155615
0.40694076038903626
0.40870910698496904
0.40804597701

0.5481874447391689
0.5484084880636605
0.5725022104332449
0.5512820512820513
0.5488505747126436
0.5515030946065429
0.5490716180371353
0.5742705570291777
0.5495137046861185
0.5517241379310345
0.5499557913351016
0.5510610079575596
0.5749336870026526
0.551945181255526
0.5503978779840849
0.5492926613616269
0.5508399646330681
0.5501768346595933
0.5506189213085765
0.5497347480106101
0.5534924845269673
0.5528293545534925
0.5526083112290009
0.5548187444739169
0.5550397877984085
0.5532714412024757
0.5658709106984969
0.5537135278514589
0.5521662245800176
0.5543766578249337
0.5545977011494253
0.5676392572944297
0.5523872679045093
0.5539345711759505
0.553050397877984
0.554155614500442
0.5559239610963749
0.6432360742705571
0.5722811671087533
0.5557029177718833
0.5565870910698497
0.5563660477453581
0.5570291777188329
0.5554818744473917
0.5705128205128205
0.5568081343943413
0.5552608311229
0.5561450044208665
0.6971706454465075
0.5583554376657824
0.5574712643678161
0.5592396109637489
0.622236958443855


0.7011494252873564
0.7009283819628647
0.7015915119363395
0.7029177718832891
0.7026967285587975
0.7038019451812555
0.7046861184792219
0.7051282051282052
0.7033598585322723
0.7053492484526968
0.7057913351016799
0.7044650751547303
0.7049071618037135
0.7035809018567639
0.7055702917771883
0.7042440318302388
0.7031388152077808
0.7024756852343059
0.7066755083996463
0.7062334217506632
0.7073386383731212
0.7080017683465959
0.7064544650751547
0.7084438549955792
0.7068965517241379
0.7071175950486295
0.7077807250221043
0.7086648983200707
0.7091069849690539
0.7082228116710876
0.7088859416445623
0.7060123784261715
0.7075596816976127
0.7126436781609196
0.712422634836428
0.7102122015915119
0.7106542882404951
0.7115384615384616
0.7119805481874447
0.7104332449160036
0.71131741821397
0.7097701149425287
0.7117595048629531
0.7095490716180372
0.7093280282935456
0.7099911582670203
0.7110963748894783
0.7122015915119363
0.7144120247568524
0.7133068081343943
0.7148541114058355
0.7130857648099027
0.7150751547303

0.8530061892130858
0.853448275862069
0.8521220159151194
0.8552166224580018
0.8547745358090185
0.8554376657824934
0.8569849690539346
0.8556587091069849
0.8549955791335102
0.8567639257294429
0.8558797524314765
0.8565428824049514
0.8572060123784262
0.8563218390804598
0.8561007957559682
0.8576480990274093
0.8598585322723253
0.8594164456233422
0.8574270557029178
0.8605216622458002
0.8583112290008842
0.8587533156498673
0.8603006189213086
0.8580901856763926
0.8589743589743589
0.8585322723253758
0.860079575596817
0.8578691423519009
0.8596374889478338
0.8591954022988506
0.8625110521662246
0.8609637488947833
0.8629531388152077
0.8616268788682582
0.8614058355437666
0.862290008841733
0.8627320954907162
0.8631741821396994
0.8640583554376657
0.863395225464191
0.8620689655172413
0.8638373121131742
0.8636162687886826
0.8618479221927497
0.861184792219275
0.8651635720601237
0.8658267020335986
0.864500442086649
0.8662687886825818
0.8647214854111406
0.8649425287356322
0.8673740053050398
0.8671529619805481

In [58]:
len(datetime_coordinate_matching.keys())/999

3.1981981981981984

In [8]:
venue_dict

{'Tokyo Dome': {0: 35.705526, 1: 139.751928, 2: 57.0},
 'Great American Ball Park': {0: 39.097213, 1: -84.506483, 2: 489.0},
 'Kauffman Stadium': {0: 39.051604, 1: -94.480149, 2: 886.0},
 'Dodger Stadium': {0: 34.073878, 1: -118.239951, 2: 501.0},
 'Nationals Park': {0: 38.872987, 1: -77.007435, 2: 23.0},
 'Oakland Coliseum': {0: 37.751619, 1: -122.200451, 2: 0.0},
 'Petco Park': {0: 32.70753, 1: -117.157056, 2: 14.0},
 'T-Mobile Park': {0: 47.591443, 1: -122.332283, 2: 17.0},
 'Tropicana Field': {0: 27.768254, 1: -82.653431, 2: 41.0},
 'Globe Life Park in Arlington': {0: 32.751164, 1: -97.082546, 2: 549.0},
 'Rogers Centre': {0: 43.641256, 1: -79.389054, 2: 268.0},
 'Target Field': {0: 44.981749, 1: -93.278026, 2: 840.0},
 'Citizens Bank Park': {0: 39.906109, 1: -75.166485, 2: 19.0},
 'Marlins Park': {0: 25.778165, 1: -80.219541, 2: 6.0},
 'Yankee Stadium': {0: 40.829586, 1: -73.926413, 2: 11.0},
 'Miller Park': {0: 43.028232, 1: -87.970966, 2: 602.0},
 'Progressive Field': {0: 41.496

In [54]:
from datetime import datetime
from dateutil import tz
from geopy.geocoders import Nominatim

def convert_UTC_to_local(row):
    venue_name = row['venue_name']
    game_datetime = row['game_datetime']
    from_zone = tz.gettz('UTC')
    
    locator = Nominatim(user_agent='myGeocoder')
    city, lat, lon, alt = venue_dict[venue_name]
#     coordinates = ', '.join([str(lat), str(lon)])
#     location = locator.reverse(coordinates)
#     print(location.raw)
#     venue_city = location.raw['address']['city']
    if city == 'Tokyo':
        to_zone = tz.gettz('Asia/Tokyo')
    elif city == 'London':
        to_zone = tz.gettz('Europe/London')
    elif city in ['San Francisco',
                        'Oakland',
                        'Phoenix',
                        'Seattle',
                        'Los Angeles',
                        'San Diego',
                        'Anaheim']:
        to_zone = tz.gettz('America/Los_Angeles')
    elif city == 'Denver':
        to_zone = tz.gettz('America/Denver')
    elif city in ['Minneapolis',
                        'Milwaukee',
                        'Chicago',
                        'St. Louis',
                        'Arlington',
                        'Kansas City',
                        'Houston',
                        'Monterrey',
                        'Omaha']:
        to_zone = tz.gettz('America/Chicago')
    elif city in ['Buffalo',
                        'Detroit',
                        'Cincinnati',
                        'Pittsburgh',
                        'Tampa Bay',
                        'Philadelphia',
                        'Atlanta',
                        'New York',
                        'Washington',
                        'Cleveland',
                        'Miami',
                        'Boston',
                        'Baltimore',
                        'Toronto',
                        'Williamsport',
                        'Dunedin',
                        'St. Petersburg']:
        to_zone = tz.gettz('America/New_York')
    else:
        raise IndexError(venue_name, city)
    utc = datetime.strptime(game_datetime, '%Y-%m-%dT%H:%M:%SZ')
    utc = utc.replace(tzinfo=from_zone)
    local = utc.astimezone(to_zone)
    print(local)
    return(local)

game_pk_df['local_datetime'] = game_pk_df.apply(lambda row: convert_UTC_to_local(row), axis=1)

2019-03-20 18:35:00+09:00
2019-03-21 18:35:00+09:00
2019-03-28 13:05:00-04:00
2019-03-28 13:05:00-04:00
2019-03-28 13:10:00-05:00
2019-03-28 15:05:00-04:00
2019-03-28 15:37:00-04:00
2019-03-28 16:00:00-04:00
2019-03-28 15:05:00-05:00
2019-03-28 13:07:00-07:00
2019-03-28 13:10:00-07:00
2019-03-28 15:10:00-05:00
2019-03-28 13:10:00-07:00
2019-03-28 16:10:00-04:00
2019-03-28 16:10:00-04:00
2019-03-28 15:15:00-05:00
2019-03-28 16:10:00-07:00
2019-03-29 19:07:00-04:00
2019-03-29 19:10:00-04:00
2019-03-29 19:10:00-04:00
2019-03-29 19:10:00-05:00
2019-03-29 19:07:00-07:00
2019-03-29 19:10:00-07:00
2019-03-29 19:10:00-07:00
2019-03-29 19:10:00-07:00
2019-03-30 13:05:00-04:00
2019-03-30 13:05:00-04:00
2019-03-30 13:10:00-05:00
2019-03-30 14:10:00-04:00
2019-03-30 13:15:00-05:00
2019-03-30 15:07:00-04:00
2019-03-30 16:05:00-04:00
2019-03-30 18:10:00-04:00
2019-03-30 18:10:00-04:00
2019-03-30 18:10:00-05:00
2019-03-30 19:05:00-05:00
2019-03-30 17:40:00-07:00
2019-03-30 18:07:00-07:00
2019-03-30 1

2019-04-25 19:10:00-04:00
2019-04-25 19:10:00-05:00
2019-04-25 18:07:00-07:00
2019-04-25 19:10:00-07:00
2019-04-26 19:05:00-04:00
2019-04-26 19:05:00-04:00
2019-04-26 19:07:00-04:00
2019-04-26 19:10:00-04:00
2019-04-26 19:10:00-04:00
2019-04-26 19:20:00-04:00
2019-04-26 19:10:00-05:00
2019-04-26 19:10:00-05:00
2019-04-26 19:10:00-05:00
2019-04-26 19:15:00-05:00
2019-04-26 19:15:00-05:00
2019-04-26 18:40:00-07:00
2019-04-26 19:10:00-07:00
2019-04-26 19:10:00-07:00
2019-04-26 19:15:00-07:00
2019-04-27 13:10:00-05:00
2019-04-27 13:15:00-05:00
2019-04-27 15:07:00-04:00
2019-04-27 16:05:00-04:00
2019-04-27 15:05:00-05:00
2019-04-27 16:05:00-04:00
2019-04-27 13:05:00-07:00
2019-04-27 18:05:00-04:00
2019-04-27 18:10:00-05:00
2019-04-27 19:10:00-04:00
2019-04-27 18:15:00-05:00
2019-04-27 19:20:00-04:00
2019-04-27 17:10:00-07:00
2019-04-27 18:10:00-07:00
2019-04-27 18:10:00-07:00
2019-04-28 13:05:00-04:00
2019-04-28 13:05:00-04:00
2019-04-28 13:07:00-04:00
2019-04-28 13:10:00-04:00
2019-04-28 1

2019-05-24 19:10:00-05:00
2019-05-24 19:15:00-05:00
2019-05-24 19:15:00-05:00
2019-05-24 18:40:00-06:00
2019-05-24 19:07:00-07:00
2019-05-24 19:07:00-07:00
2019-05-24 19:15:00-07:00
2019-05-25 13:10:00-05:00
2019-05-25 13:15:00-05:00
2019-05-25 13:20:00-05:00
2019-05-25 15:07:00-04:00
2019-05-25 13:05:00-07:00
2019-05-25 16:05:00-04:00
2019-05-25 13:07:00-07:00
2019-05-25 16:10:00-04:00
2019-05-25 15:10:00-05:00
2019-05-25 16:10:00-04:00
2019-05-25 19:15:00-04:00
2019-05-25 18:15:00-05:00
2019-05-25 18:15:00-05:00
2019-05-25 19:10:00-06:00
2019-05-25 19:07:00-07:00
2019-05-26 13:07:00-04:00
2019-05-26 13:10:00-04:00
2019-05-26 13:10:00-04:00
2019-05-26 13:35:00-04:00
2019-05-26 13:35:00-04:00
2019-05-26 13:10:00-05:00
2019-05-26 13:10:00-05:00
2019-05-26 13:10:00-05:00
2019-05-26 13:15:00-05:00
2019-05-26 13:20:00-05:00
2019-05-26 13:10:00-06:00
2019-05-26 13:05:00-07:00
2019-05-26 13:07:00-07:00
2019-05-26 13:07:00-07:00
2019-05-26 18:05:00-05:00
2019-05-27 13:05:00-04:00
2019-05-27 1

2019-06-21 19:10:00-05:00
2019-06-21 19:15:00-05:00
2019-06-21 19:15:00-05:00
2019-06-21 18:40:00-07:00
2019-06-21 19:07:00-07:00
2019-06-21 19:10:00-07:00
2019-06-21 19:10:00-07:00
2019-06-22 13:15:00-05:00
2019-06-22 13:15:00-05:00
2019-06-22 13:20:00-05:00
2019-06-22 16:05:00-04:00
2019-06-22 16:05:00-04:00
2019-06-22 16:05:00-04:00
2019-06-22 13:07:00-07:00
2019-06-22 15:10:00-05:00
2019-06-22 13:10:00-07:00
2019-06-22 16:10:00-04:00
2019-06-22 19:15:00-04:00
2019-06-22 19:15:00-04:00
2019-06-22 16:15:00-07:00
2019-06-22 20:05:00-05:00
2019-06-22 19:10:00-07:00
2019-06-23 13:05:00-04:00
2019-06-23 13:05:00-04:00
2019-06-23 13:10:00-04:00
2019-06-23 13:35:00-04:00
2019-06-23 13:35:00-04:00
2019-06-23 14:05:00-04:00
2019-06-23 13:10:00-05:00
2019-06-23 13:15:00-05:00
2019-06-23 13:20:00-05:00
2019-06-23 14:05:00-05:00
2019-06-23 13:07:00-07:00
2019-06-23 13:10:00-07:00
2019-06-23 13:10:00-07:00
2019-06-23 13:10:00-07:00
2019-06-23 18:05:00-05:00
2019-06-24 19:05:00-04:00
2019-06-24 1

2019-07-21 13:10:00-07:00
2019-07-21 19:05:00-04:00
2019-07-22 19:05:00-04:00
2019-07-22 19:05:00-04:00
2019-07-22 19:07:00-04:00
2019-07-22 19:10:00-04:00
2019-07-22 19:10:00-05:00
2019-07-22 19:10:00-05:00
2019-07-22 19:10:00-05:00
2019-07-22 19:10:00-05:00
2019-07-22 18:40:00-07:00
2019-07-22 18:45:00-07:00
2019-07-22 19:10:00-07:00
2019-07-23 19:05:00-04:00
2019-07-23 19:05:00-04:00
2019-07-23 19:07:00-04:00
2019-07-23 19:10:00-04:00
2019-07-23 19:10:00-04:00
2019-07-23 19:10:00-04:00
2019-07-23 19:20:00-04:00
2019-07-23 19:10:00-05:00
2019-07-23 19:10:00-05:00
2019-07-23 19:10:00-05:00
2019-07-23 19:10:00-05:00
2019-07-23 18:40:00-07:00
2019-07-23 18:45:00-07:00
2019-07-23 19:10:00-07:00
2019-07-23 19:10:00-07:00
2019-07-24 12:10:00-04:00
2019-07-24 13:10:00-04:00
2019-07-24 13:10:00-05:00
2019-07-24 13:10:00-05:00
2019-07-24 12:40:00-07:00
2019-07-24 12:40:00-07:00
2019-07-24 12:45:00-07:00
2019-07-24 19:05:00-04:00
2019-07-24 19:05:00-04:00
2019-07-24 19:07:00-04:00
2019-07-24 1

...

2020-08-02 13:10:00-07:00
2020-08-02 19:08:00-04:00
2020-08-03 18:40:00-04:00
2020-08-03 19:10:00-04:00
2020-08-03 19:10:00-05:00
2020-08-03 19:10:00-05:00
2020-08-03 19:15:00-05:00
2020-08-03 18:40:00-06:00
2020-08-03 18:10:00-07:00
2020-08-03 18:10:00-07:00
2020-08-04 13:10:00-05:00
2020-08-04 18:10:00-04:00
2020-08-04 18:40:00-04:00
2020-08-04 19:05:00-04:00
2020-08-04 19:10:00-04:00
2020-08-04 19:10:00-04:00
2020-08-04 19:10:00-05:00
2020-08-04 19:15:00-05:00
2020-08-04 18:40:00-06:00
2020-08-04 18:10:00-07:00
2020-08-04 18:10:00-07:00
2020-08-04 18:10:00-07:00
2020-08-04 19:10:00-07:00
2020-08-04 23:33:00-04:00
2020-08-05 18:05:00-04:00
2020-08-05 18:40:00-04:00
2020-08-05 19:05:00-04:00
2020-08-05 19:10:00-04:00
2020-08-05 19:10:00-04:00
2020-08-05 19:10:00-04:00
2020-08-05 19:35:00-04:00
2020-08-05 19:05:00-05:00
2020-08-05 19:10:00-05:00
2020-08-05 18:40:00-06:00
2020-08-05 18:10:00-07:00
2020-08-05 18:10:00-07:00
2020-08-05 18:10:00-07:00
2020-08-05 18:40:00-07:00
2020-08-06 1

2020-08-30 13:10:00-04:00
2020-08-30 13:10:00-04:00
2020-08-30 13:10:00-04:00
2020-08-30 13:35:00-04:00
2020-08-30 13:10:00-05:00
2020-08-30 13:10:00-05:00
2020-08-30 13:10:00-05:00
2020-08-30 13:15:00-05:00
2020-08-30 13:35:00-05:00
2020-08-30 15:07:00-04:00
2020-08-30 13:10:00-06:00
2020-08-30 13:10:00-07:00
2020-08-30 13:10:00-07:00
2020-08-30 19:08:00-04:00
2020-08-31 14:07:00-04:00
2020-08-31 13:10:00-07:00
2020-08-31 18:40:00-04:00
2020-08-31 19:05:00-04:00
2020-08-31 19:05:00-04:00
2020-08-31 19:30:00-04:00
2020-08-31 18:40:00-05:00
2020-08-31 19:05:00-05:00
2020-08-31 19:10:00-05:00
2020-08-31 19:40:00-06:00
2020-09-01 18:40:00-04:00
2020-09-01 18:40:00-04:00
2020-09-01 19:05:00-04:00
2020-09-01 19:05:00-04:00
2020-09-01 19:05:00-04:00
2020-09-01 19:30:00-04:00
2020-09-01 19:35:00-04:00
2020-09-01 18:40:00-05:00
2020-09-01 19:05:00-05:00
2020-09-01 19:10:00-05:00
2020-09-01 19:10:00-05:00
2020-09-01 18:40:00-06:00
2020-09-01 18:40:00-07:00
2020-09-01 18:40:00-07:00
2020-09-02 1

2021-04-06 16:05:00-04:00
2021-04-06 13:07:00-07:00
2021-04-06 18:35:00-04:00
2021-04-06 18:40:00-04:00
2021-04-06 18:40:00-04:00
2021-04-06 19:05:00-04:00
2021-04-06 19:10:00-04:00
2021-04-06 18:40:00-05:00
2021-04-06 19:05:00-05:00
2021-04-06 18:40:00-06:00
2021-04-06 18:40:00-07:00
2021-04-06 19:10:00-07:00
2021-04-06 19:10:00-07:00
2021-04-07 12:05:00-04:00
2021-04-07 12:35:00-04:00
2021-04-07 13:10:00-04:00
2021-04-07 13:10:00-04:00
2021-04-07 13:10:00-04:00
2021-04-07 13:05:00-05:00
2021-04-07 13:20:00-05:00
2021-04-07 12:37:00-07:00
2021-04-07 16:05:00-04:00
2021-04-07 13:10:00-07:00
2021-04-07 16:10:00-04:00
2021-04-07 13:10:00-07:00
2021-04-07 18:35:00-04:00
2021-04-07 18:40:00-06:00
2021-04-08 13:10:00-04:00
2021-04-08 13:35:00-04:00
2021-04-08 15:05:00-04:00
2021-04-08 13:10:00-06:00
2021-04-08 15:10:00-05:00
2021-04-08 15:10:00-05:00
2021-04-08 15:15:00-05:00
2021-04-08 19:07:00-04:00
2021-04-08 19:10:00-05:00
2021-04-09 15:10:00-04:00
2021-04-09 13:10:00-07:00
2021-04-09 1

2021-05-11 12:45:00-07:00
2021-05-11 18:10:00-04:00
2021-05-11 18:35:00-04:00
2021-05-11 19:05:00-04:00
2021-05-11 19:10:00-04:00
2021-05-11 19:10:00-04:00
2021-05-11 19:10:00-04:00
2021-05-11 19:10:00-04:00
2021-05-11 19:20:00-04:00
2021-05-11 18:40:00-05:00
2021-05-11 19:10:00-05:00
2021-05-11 19:10:00-05:00
2021-05-11 18:40:00-06:00
2021-05-11 18:40:00-07:00
2021-05-11 19:10:00-07:00
2021-05-12 12:10:00-04:00
2021-05-12 12:35:00-04:00
2021-05-12 13:10:00-04:00
2021-05-12 13:10:00-06:00
2021-05-12 19:05:00-04:00
2021-05-12 19:10:00-04:00
2021-05-12 19:10:00-04:00
2021-05-12 19:10:00-04:00
2021-05-12 19:20:00-04:00
2021-05-12 18:40:00-05:00
2021-05-12 19:10:00-05:00
2021-05-12 19:10:00-05:00
2021-05-12 18:40:00-07:00
2021-05-12 19:10:00-07:00
2021-05-13 12:20:00-04:00
2021-05-13 13:05:00-04:00
2021-05-13 13:10:00-04:00
2021-05-13 12:40:00-05:00
2021-05-13 13:10:00-05:00
2021-05-13 18:35:00-04:00
2021-05-13 19:10:00-04:00
2021-05-13 19:10:00-04:00
2021-05-13 19:10:00-05:00
2021-05-13 1

2021-06-09 19:10:00-04:00
2021-06-09 19:10:00-04:00
2021-06-09 19:10:00-04:00
2021-06-09 19:10:00-04:00
2021-06-09 19:10:00-05:00
2021-06-09 19:10:00-05:00
2021-06-09 19:15:00-05:00
2021-06-09 18:38:00-07:00
2021-06-10 12:35:00-04:00
2021-06-10 12:35:00-04:00
2021-06-10 13:05:00-04:00
2021-06-10 13:10:00-04:00
2021-06-10 19:05:00-04:00
2021-06-10 19:10:00-04:00
2021-06-10 19:10:00-04:00
2021-06-10 19:10:00-05:00
2021-06-10 19:10:00-05:00
2021-06-10 18:40:00-07:00
2021-06-11 13:20:00-05:00
2021-06-11 19:05:00-04:00
2021-06-11 19:10:00-04:00
2021-06-11 19:10:00-04:00
2021-06-11 19:10:00-04:00
2021-06-11 19:10:00-04:00
2021-06-11 19:10:00-04:00
2021-06-11 19:10:00-04:00
2021-06-11 19:10:00-04:00
2021-06-11 19:10:00-05:00
2021-06-11 19:10:00-05:00
2021-06-11 18:40:00-07:00
2021-06-11 18:40:00-07:00
2021-06-11 19:10:00-07:00
2021-06-12 16:05:00-04:00
2021-06-12 13:07:00-07:00
2021-06-12 16:10:00-04:00
2021-06-12 16:10:00-04:00
2021-06-12 15:10:00-05:00
2021-06-12 16:10:00-04:00
2021-06-12 1

In [55]:
game_pk_df['local_datetime'] = game_pk_df['local_datetime'].apply(lambda x: x.strftime("%Y-%m-%dT%H:%M:%S"))

In [56]:
game_pk_df

,game_id,game_datetime,game_date,game_type,status,away_name,home_name,away_id,home_id,doubleheader,game_num,home_probable_pitcher,away_probable_pitcher,home_pitcher_note,away_pitcher_note,away_score,home_score,current_inning,inning_state,venue_id,venue_name,winning_team,losing_team,winning_pitcher,losing_pitcher,save_pitcher,summary,local_datetime
0,566083,2019-03-20T09:35:00Z,2019-03-20,R,Final,Seattle Mariners,Oakland Athletics,136,133,N,1,Mike Fiers,Marco Gonzales,"Fiers, who will be making the first Opening Da...",The 27-year-old gets his first Opening Day sta...,9,7,9,Bottom,2397,Tokyo Dome,Seattle Mariners,Oakland Athletics,Marco Gonzales,Mike Fiers,Hunter Strickland,2019-03-20 - Seattle Mariners (9) @ Oakland At...,2019-03-20T18:35:00
1,566084,2019-03-21T09:35:00Z,2019-03-21,R,Final,Seattle Mariners,Oakland Athletics,136,133,N,1,Marco Estrada,Yusei Kikuchi,Estrada is seeking a bounce-back campaign as h...,After signing with the Mariners as a free agen...,5,4,12,Bottom,2397,Tokyo Dome,Seattle Mariners,Oakland Athletics,Zac Rosscup,Ryan Buchter,Hunter Strickland,2019-03-21 - Seattle Mariners (5) @ Oakland At...,2019-03-21T18:35:00
15,567456,2019-03-28T17:05:00Z,2019-03-28,R,Final,Baltimore Orioles,New York Yankees,110,147,N,1,Masahiro Tanaka,Andrew Cashner,Tanaka earns his fourth career Opening Day sta...,"Cashner replaces Alex Cobb, who suffered a str...",2,7,9,Top,3313,Yankee Stadium,New York Yankees,Baltimore Orioles,Masahiro Tanaka,Andrew Cashner,None,2019-03-28 - Baltimore Orioles (2) @ New York ...,2019-03-28T13:05:00
5,565895,2019-03-28T17:05:00Z,2019-03-28,R,Final,New York Mets,Washington Nationals,121,120,N,1,Max Scherzer,Jacob deGrom,Scherzer earns his fourth Opening Day assignme...,The reigning National League Cy Young Award wi...,2,0,9,Bottom,3309,Nationals Park,New York Mets,Washington Nationals,Jacob deGrom,Max Scherzer,Edwin Diaz,2019-03-28 - New York Mets (2) @ Washington Na...,2019-03-28T13:05:00
16,567552,2019-03-28T18:10:00Z,2019-03-28,R,Final,St. Louis Cardinals,Milwaukee Brewers,138,158,N,1,Jhoulys Chacin,Miles Mikolas,This is Chacin’s third career Opening Day assi...,"Mikolas, who signed a four-year extension in l...",4,5,9,Top,32,Miller Park,Milwaukee Brewers,St. Louis Cardinals,Jhoulys Chacin,Miles Mikolas,Josh Hader,2019-03-28 - St. Louis Cardinals (4) @ Milwauk...,2019-03-28T13:10:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4513,633435,2021-06-30T23:10:00Z,2021-06-30,R,Completed Early: Rain,San Diego Padres,Cincinnati Reds,135,113,N,1,Vladimir Gutierrez,Joe Musgrove,,,7,5,6,Top,2602,Great American Ball Park,NaN,NaN,NaN,NaN,NaN,2021-06-30 - San Diego Padres @ Cincinnati Red...,2021-06-30T19:10:00
4514,633437,2021-06-30T23:20:00Z,2021-06-30,R,Final,New York Mets,Atlanta Braves,121,144,N,1,Max Fried,David Peterson,,,2,20,9,Top,4705,Truist Park,Atlanta Braves,New York Mets,Max Fried,David Peterson,None,2021-06-30 - New York Mets (2) @ Atlanta Brave...,2021-06-30T19:20:00
4519,633479,2021-07-01T00:10:00Z,2021-06-30,R,Final,Minnesota Twins,Chicago White Sox,142,145,N,1,Dylan Cease,Bailey Ober,,,3,13,9,Top,4,Guaranteed Rate Field,Chicago White Sox,Minnesota Twins,Dylan Cease,Bailey Ober,None,2021-06-30 - Minnesota Twins (3) @ Chicago Whi...,2021-06-30T19:10:00
4521,633485,2021-07-01T00:10:00Z,2021-06-30,R,Final,Baltimore Orioles,Houston Astros,110,117,N,1,Luis Garcia,Matt Harvey,,,5,2,9,Bottom,2392,Minute Maid Park,Baltimore Orioles,Houston Astros,Tanner Scott,Luis Garcia,Cole Sulser,2021-06-30 - Baltimore Orioles (5) @ Houston A...,2021-06-30T19:10:00


In [57]:
datetime_coordinate_matching = {}
for index, row in game_pk_df.iterrows():
    city, lat, lon, alt = venue_dict[row['venue_name']]
    dc_datetime = row['local_datetime']
    if dc_datetime in datetime_coordinate_matching.keys():
        datetime_coordinate_matching[dc_datetime].append(','.join([str(lat), str(lon)]))
    else:
        datetime_coordinate_matching[dc_datetime] = [','.join([str(lat), str(lon)])]
#     print(index/4524)

In [139]:
game_pk_df['venue_name']

0                     Tokyo Dome
1                     Tokyo Dome
15                Yankee Stadium
5                 Nationals Park
16                   Miller Park
                  ...           
4513    Great American Ball Park
4514                 Truist Park
4519       Guaranteed Rate Field
4521            Minute Maid Park
4516            Oakland Coliseum
Name: venue_name, Length: 4524, dtype: object

In [146]:
game_pk_df['coordinates'] = game_pk_df['venue_name'].apply(lambda x: ','.join(venue_dict[x][1:3]))

In [61]:
for key, value in datetime_coordinate_matching.items():
    print(value)

['35.705526,139.751928']
['35.705526,139.751928']
['40.829586,-73.926413', '38.872987,-77.007435']
['43.028232,-87.970966', '32.70753,-117.157056', '34.073878,-118.239951']
['39.906109,-75.166485', '32.751164,-97.082546']
['43.641256,-79.389054']
['27.768254,-82.653431']
['37.751619,-122.200451']
['44.981749,-93.278026']
['25.778165,-80.219541', '39.097213,-84.506483', '47.591443,-122.332283']
['39.051604,-94.480149']
['43.641256,-79.389054', '37.751619,-122.200451']
['25.778165,-80.219541', '27.768254,-82.653431', '43.028232,-87.970966', '47.591443,-122.332283', '32.70753,-117.157056', '34.073878,-118.239951']
['40.829586,-73.926413', '38.872987,-77.007435']
['44.981749,-93.278026']
['39.097213,-84.506483']
['39.051604,-94.480149']
['43.641256,-79.389054']
['39.906109,-75.166485']
['25.778165,-80.219541', '27.768254,-82.653431', '43.028232,-87.970966', '47.591443,-122.332283', '34.073878,-118.239951']
['32.751164,-97.082546']
['32.70753,-117.157056']
['37.751619,-122.200451']
['40.829

['39.283944,-76.621572']
['33.44542,-112.066793']
['41.948314,-87.655397']
['42.339063,-83.048627']
['39.283944,-76.621572', '39.051604,-94.480149', '37.778324,-122.389221']
['25.778165,-80.219541', '33.89127,-84.4681', '43.028232,-87.970966', '44.981749,-93.278026', '39.756175,-104.99413', '47.591443,-122.332283', '33.44542,-112.066793', '33.80029,-117.882685']
['41.948314,-87.655397']
['39.906109,-75.166485']
['40.829586,-73.926413']
['39.097213,-84.506483', '39.756175,-104.99413']
['33.89127,-84.4681', '43.028232,-87.970966', '44.981749,-93.278026']
['41.948314,-87.655397']
['32.70753,-117.157056', '47.591443,-122.332283']
['44.981749,-93.278026']
['39.097213,-84.506483', '33.44542,-112.066793', '37.751619,-122.200451', '32.70753,-117.157056']
['27.768254,-82.653431', '39.756175,-104.99413']
['38.872987,-77.007435']
['25.778165,-80.219541', '33.89127,-84.4681', '43.028232,-87.970966', '47.591443,-122.332283']
['41.948314,-87.655397']
['42.339063,-83.048627']
['38.872987,-77.007435']

['37.778324,-122.389221']
['38.622622,-90.192841']
['43.028232,-87.970966', '39.051604,-94.480149', '41.829892,-87.633703', '39.756175,-104.99413']
['37.751619,-122.200451']
['38.872987,-77.007435']
['42.881306,-78.874278']
['40.757134,-73.84584', '41.496005,-81.685326', '29.757041,-95.355429', '34.073878,-118.239951', '47.591443,-122.332283']
['33.89127,-84.4681']
['37.778324,-122.389221']
['41.496005,-81.685326']
['37.778324,-122.389221']
['42.881306,-78.874278']
['40.757134,-73.84584', '29.757041,-95.355429', '47.591443,-122.332283']
['33.89127,-84.4681']
['32.70753,-117.157056']
['39.756175,-104.99413']
['33.80029,-117.882685']
['38.872987,-77.007435', '40.829586,-73.926413', '39.283944,-76.621572', '40.446835,-80.005683', '41.948314,-87.655397', '32.751164,-97.082546']
['33.89127,-84.4681']
['39.051604,-94.480149', '29.757041,-95.355429', '32.70753,-117.157056', '47.591443,-122.332283']
['39.756175,-104.99413', '33.44542,-112.066793']
['33.80029,-117.882685']
['37.778324,-122.3892

In [126]:
from api_keys import visual_crossing
import requests

weather_df = pd.DataFrame()

for key, value in datetime_coordinate_matching.items():
    url_locations = '|'.join(value)
    URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?&aggregateHours=1&startDateTime={key}&endDateTime={key}&unitGroup=us&contentType=json&location={url_locations}&key={visual_crossing}'
    try:
        response = requests.get(URL)
        data = response.json()
        locations = list(data['locations'].keys())
        for each in locations:
            values = data['locations'][each]['values'][0]
            weather_df = weather_df.append({**{'coordinates': each}, **values}, ignore_index=True)
    except:
        raise IndexError(key, value, response)
#     [list(response.json()['locations'].keys())[0]]['values'][0]
#     break
#     data = data.decode('utf-8').splitlines()
#     if data[0][:7] != 'Address':
#         print(data)
#         break
#     data = dict(zip(str.split(data[0], ','), str.split(data[1], ',')))
#     weather_df = weather_df.append(data, ignore_index=True)
    
    
# return(url_datetime, data)

# game_pk_df.apply(lambda row: vc_weather(row))



In [135]:
weather_df['datetimeStr'] = weather_df['datetimeStr'].apply(lambda x: x[:-6])
weather_df

,cloudcover,conditions,coordinates,datetime,datetimeStr,dew,heatindex,humidity,info,maxt,mint,precip,precipcover,sealevelpressure,snow,snowdepth,solarenergy,solarradiation,temp,visibility,wdir,weathertype,wgust,windchill,wspd
0,0.0,Clear,"35.705526,139.751928",1.553107e+12,2019-03-20T18:35:00,39.5,None,43.51,None,61.9,61.9,0.00,None,1018.1,None,None,None,None,61.9,11.4,180.0,,None,None,8.1
1,0.0,Clear,"35.705526,139.751928",1.553193e+12,2019-03-21T18:35:00,57.8,None,78.49,None,64.6,64.6,0.00,None,1002.2,None,None,None,None,64.6,11.4,190.0,,None,None,17.2
2,46.0,Partially cloudy,"38.872987,-77.007435",1.553778e+12,2019-03-28T13:05:00,34.7,None,40.25,None,58.8,58.8,0.00,None,1027.1,None,None,None,None,58.8,9.9,171.0,,None,None,11.1
3,39.4,Partially cloudy,"40.829586,-73.926413",1.553778e+12,2019-03-28T13:05:00,28.5,None,43.74,None,49.7,49.7,0.00,None,1027.8,None,None,None,None,49.7,9.9,301.0,,None,46.5,7.6
4,53.1,Partially cloudy,"32.70753,-117.157056",1.553779e+12,2019-03-28T13:10:00,53.4,None,62.13,None,66.8,66.8,0.00,None,1023.4,None,None,None,None,66.8,9.9,288.0,,None,None,14.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4518,90.7,Overcast,"39.097213,-84.506483",1.625080e+12,2021-06-30T19:10:00,68.0,None,82.30,None,73.7,73.7,0.00,None,1017.0,None,None,None,None,73.7,9.9,65.0,,None,None,4.3
4519,93.7,Overcast,"29.757041,-95.355429",1.625080e+12,2021-06-30T19:10:00,73.7,91.5,70.28,None,84.4,84.4,0.00,None,1016.1,None,None,None,None,84.4,9.9,160.0,,None,None,9.5
4520,78.9,"Rain, Overcast","41.829892,-87.633703",1.625080e+12,2021-06-30T19:10:00,66.8,None,78.16,None,74.0,74.0,0.01,None,1015.4,None,None,None,None,74.0,9.9,196.0,,None,None,5.6
4521,46.4,Partially cloudy,"33.89127,-84.4681",1.625081e+12,2021-06-30T19:20:00,71.8,86.0,72.43,None,81.5,81.5,0.00,None,1018.6,None,None,None,None,81.5,9.9,54.0,"Lightning Without Thunder, Thunderstorm",None,None,4.0


Important to save since this actually cost money

In [128]:
weather_df.to_csv('weather_df')

In [129]:
df_filtered

,Unnamed: 0,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bottom
0,1,NaN,737138,NaN,NaN,NaN,"Fiers, Mike",543829,571666,field_out,hit_into_play,NaN,NaN,NaN,NaN,NaN,Dee Gordon lines out sharply to right fielder ...,R,L,R,OAK,SEA,X,9.0,line_drive,1,0,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,199.00,109.62,NaN,NaN,460026.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,566083,571666,460026.0,621566.0,595777.0,656305.0,543760.0,640461.0,657656.0,572039.0,NaN,NaN,NaN,0.0,1.0,0.0,0.0,NaN,1,2,NaN,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0.022,-0.285,0
1,5,NaN,737138,NaN,NaN,NaN,"Fiers, Mike",571745,571666,field_out,hit_into_play,NaN,NaN,NaN,NaN,NaN,Mitch Haniger pops out to first baseman Matt O...,R,R,R,OAK,SEA,X,3.0,popup,0,2,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,166.04,156.26,NaN,NaN,460026.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,566083,571666,460026.0,621566.0,595777.0,656305.0,543760.0,640461.0,657656.0,572039.0,NaN,NaN,NaN,0.0,1.0,0.0,0.0,NaN,2,4,NaN,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0.015,-0.106,0
2,6,NaN,737138,NaN,NaN,NaN,"Fiers, Mike",457803,571666,single,hit_into_play,NaN,NaN,NaN,NaN,NaN,Jay Bruce singles on a sharp line drive to sho...,R,L,R,OAK,SEA,X,6.0,line_drive,0,0,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,161.99,138.51,NaN,NaN,460026.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,566083,571666,460026.0,621566.0,595777.0,656305.0,543760.0,640461.0,657656.0,572039.0,NaN,NaN,NaN,0.9,1.0,1.0,0.0,NaN,3,1,NaN,0,0,0,0,0,0,0,0,NaN,NaN,NaN,-0.012,0.128,0
3,11,NaN,737138,NaN,NaN,NaN,"Fiers, Mike",429665,571666,strikeout,swinging_strike,NaN,NaN,NaN,NaN,NaN,Edwin Encarnacion strikes out swinging.,R,R,R,OAK,SEA,S,2.0,NaN,2,2,2019,NaN,NaN,NaN,NaN,NaN,457803.0,NaN,2,1,NaN,NaN,NaN,NaN,460026.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,566083,571666,460026.0,621566.0,595777.0,656305.0,543760.0,640461.0,657656.0,572039.0,NaN,NaN,NaN,0.0,1.0,0.0,0.0,NaN,4,5,NaN,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0.032,-0.275,0
4,14,NaN,737138,NaN,NaN,NaN,"Gonzales, Marco",657656,594835,strikeout,swinging_strike,NaN,NaN,NaN,NaN,NaN,Ramon Laureano strikes out swinging.,R,R,L,OAK,SEA,S,2.0,NaN,0,2,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,NaN,NaN,553882.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,566083,594835,553882.0,457803.0,543829.0,592387.0,542921.0,570267.0,571745.0,400085.0,NaN,NaN,NaN,0.0,1.0,0.0,0.0,NaN,5,3,NaN,0,0,0,0,0,0,0,0,NaN,NaN,NaN,-0.021,-0.157,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340939,1345219,FF,737971,95.4,-0.32,6.44,"Karinchak, James",606992,675916,field_out,hit_into_play,NaN,NaN,NaN,NaN,14.0,Eric Haase flies out to right fielder Harold R...,R,R,R,CLE,DET,X,9.0,fly_ball,0,

In [130]:
game_pk_df

,game_id,game_datetime,game_date,game_type,status,away_name,home_name,away_id,home_id,doubleheader,game_num,home_probable_pitcher,away_probable_pitcher,home_pitcher_note,away_pitcher_note,away_score,home_score,current_inning,inning_state,venue_id,venue_name,winning_team,losing_team,winning_pitcher,losing_pitcher,save_pitcher,summary,local_datetime
0,566083,2019-03-20T09:35:00Z,2019-03-20,R,Final,Seattle Mariners,Oakland Athletics,136,133,N,1,Mike Fiers,Marco Gonzales,"Fiers, who will be making the first Opening Da...",The 27-year-old gets his first Opening Day sta...,9,7,9,Bottom,2397,Tokyo Dome,Seattle Mariners,Oakland Athletics,Marco Gonzales,Mike Fiers,Hunter Strickland,2019-03-20 - Seattle Mariners (9) @ Oakland At...,2019-03-20T18:35:00
1,566084,2019-03-21T09:35:00Z,2019-03-21,R,Final,Seattle Mariners,Oakland Athletics,136,133,N,1,Marco Estrada,Yusei Kikuchi,Estrada is seeking a bounce-back campaign as h...,After signing with the Mariners as a free agen...,5,4,12,Bottom,2397,Tokyo Dome,Seattle Mariners,Oakland Athletics,Zac Rosscup,Ryan Buchter,Hunter Strickland,2019-03-21 - Seattle Mariners (5) @ Oakland At...,2019-03-21T18:35:00
15,567456,2019-03-28T17:05:00Z,2019-03-28,R,Final,Baltimore Orioles,New York Yankees,110,147,N,1,Masahiro Tanaka,Andrew Cashner,Tanaka earns his fourth career Opening Day sta...,"Cashner replaces Alex Cobb, who suffered a str...",2,7,9,Top,3313,Yankee Stadium,New York Yankees,Baltimore Orioles,Masahiro Tanaka,Andrew Cashner,None,2019-03-28 - Baltimore Orioles (2) @ New York ...,2019-03-28T13:05:00
5,565895,2019-03-28T17:05:00Z,2019-03-28,R,Final,New York Mets,Washington Nationals,121,120,N,1,Max Scherzer,Jacob deGrom,Scherzer earns his fourth Opening Day assignme...,The reigning National League Cy Young Award wi...,2,0,9,Bottom,3309,Nationals Park,New York Mets,Washington Nationals,Jacob deGrom,Max Scherzer,Edwin Diaz,2019-03-28 - New York Mets (2) @ Washington Na...,2019-03-28T13:05:00
16,567552,2019-03-28T18:10:00Z,2019-03-28,R,Final,St. Louis Cardinals,Milwaukee Brewers,138,158,N,1,Jhoulys Chacin,Miles Mikolas,This is Chacin’s third career Opening Day assi...,"Mikolas, who signed a four-year extension in l...",4,5,9,Top,32,Miller Park,Milwaukee Brewers,St. Louis Cardinals,Jhoulys Chacin,Miles Mikolas,Josh Hader,2019-03-28 - St. Louis Cardinals (4) @ Milwauk...,2019-03-28T13:10:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4513,633435,2021-06-30T23:10:00Z,2021-06-30,R,Completed Early: Rain,San Diego Padres,Cincinnati Reds,135,113,N,1,Vladimir Gutierrez,Joe Musgrove,,,7,5,6,Top,2602,Great American Ball Park,NaN,NaN,NaN,NaN,NaN,2021-06-30 - San Diego Padres @ Cincinnati Red...,2021-06-30T19:10:00
4514,633437,2021-06-30T23:20:00Z,2021-06-30,R,Final,New York Mets,Atlanta Braves,121,144,N,1,Max Fried,David Peterson,,,2,20,9,Top,4705,Truist Park,Atlanta Braves,New York Mets,Max Fried,David Peterson,None,2021-06-30 - New York Mets (2) @ Atlanta Brave...,2021-06-30T19:20:00
4519,633479,2021-07-01T00:10:00Z,2021-06-30,R,Final,Minnesota Twins,Chicago White Sox,142,145,N,1,Dylan Cease,Bailey Ober,,,3,13,9,Top,4,Guaranteed Rate Field,Chicago White Sox,Minnesota Twins,Dylan Cease,Bailey Ober,None,2021-06-30 - Minnesota Twins (3) @ Chicago Whi...,2021-06-30T19:10:00
4521,633485,2021-07-01T00:10:00Z,2021-06-30,R,Final,Baltimore Orioles,Houston Astros,110,117,N,1,Luis Garcia,Matt Harvey,,,5,2,9,Bottom,2392,Minute Maid Park,Baltimore Orioles,Houston Astros,Tanner Scott,Luis Garcia,Cole Sulser,2021-06-30 - Baltimore Orioles (5) @ Houston A...,2021-06-30T19:10:00


In [148]:
games_and_weather = pd.merge(
    game_pk_df,
    weather_df,
    how="outer",
    on=None,
    left_on=['local_datetime', 'coordinates'],
    right_on=['datetimeStr', 'coordinates'],
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_gpk", "_acw"),
    copy=True,
    indicator=False,
    validate=None,
)

In [149]:
games_and_weather

,game_id,game_datetime,game_date,game_type,status,away_name,home_name,away_id,home_id,doubleheader,game_num,home_probable_pitcher,away_probable_pitcher,home_pitcher_note,away_pitcher_note,away_score,home_score,current_inning,inning_state,venue_id,venue_name,winning_team,losing_team,winning_pitcher,losing_pitcher,save_pitcher,summary,local_datetime,coordinates,cloudcover,conditions,datetime,datetimeStr,dew,heatindex,humidity,info,maxt,mint,precip,precipcover,sealevelpressure,snow,snowdepth,solarenergy,solarradiation,temp,visibility,wdir,weathertype,wgust,windchill,wspd
0,566083,2019-03-20T09:35:00Z,2019-03-20,R,Final,Seattle Mariners,Oakland Athletics,136,133,N,1,Mike Fiers,Marco Gonzales,"Fiers, who will be making the first Opening Da...",The 27-year-old gets his first Opening Day sta...,9,7,9,Bottom,2397,Tokyo Dome,Seattle Mariners,Oakland Athletics,Marco Gonzales,Mike Fiers,Hunter Strickland,2019-03-20 - Seattle Mariners (9) @ Oakland At...,2019-03-20T18:35:00,"35.705526,139.751928",0.0,Clear,1.553107e+12,2019-03-20T18:35:00,39.5,None,43.51,None,61.9,61.9,0.00,None,1018.1,None,None,None,None,61.9,11.4,180.0,,None,None,8.1
1,566084,2019-03-21T09:35:00Z,2019-03-21,R,Final,Seattle Mariners,Oakland Athletics,136,133,N,1,Marco Estrada,Yusei Kikuchi,Estrada is seeking a bounce-back campaign as h...,After signing with the Mariners as a free agen...,5,4,12,Bottom,2397,Tokyo Dome,Seattle Mariners,Oakland Athletics,Zac Rosscup,Ryan Buchter,Hunter Strickland,2019-03-21 - Seattle Mariners (5) @ Oakland At...,2019-03-21T18:35:00,"35.705526,139.751928",0.0,Clear,1.553193e+12,2019-03-21T18:35:00,57.8,None,78.49,None,64.6,64.6,0.00,None,1002.2,None,None,None,None,64.6,11.4,190.0,,None,None,17.2
2,567456,2019-03-28T17:05:00Z,2019-03-28,R,Final,Baltimore Orioles,New York Yankees,110,147,N,1,Masahiro Tanaka,Andrew Cashner,Tanaka earns his fourth career Opening Day sta...,"Cashner replaces Alex Cobb, who suffered a str...",2,7,9,Top,3313,Yankee Stadium,New York Yankees,Baltimore Orioles,Masahiro Tanaka,Andrew Cashner,None,2019-03-28 - Baltimore Orioles (2) @ New York ...,2019-03-28T13:05:00,"40.829586,-73.926413",39.4,Partially cloudy,1.553778e+12,2019-03-28T13:05:00,28.5,None,43.74,None,49.7,49.7,0.00,None,1027.8,None,None,None,None,49.7,9.9,301.0,,None,46.5,7.6
3,565895,2019-03-28T17:05:00Z,2019-03-28,R,Final,New York Mets,Washington Nationals,121,120,N,1,Max Scherzer,Jacob deGrom,Scherzer earns his fourth Opening Day assignme...,The reigning National League Cy Young Award wi...,2,0,9,Bottom,3309,Nationals Park,New York Mets,Washington Nationals,Jacob deGrom,Max Scherzer,Edwin Diaz,2019-03-28 - New York Mets (2) @ Washington Na...,2019-03-28T13:05:00,"38.872987,-77.007435",46.0,Partially cloudy,1.553778e+12,2019-03-28T13:05:00,34.7,None,40.25,None,58.8,58.8,0.00,None,1027.1,None,None,None,None,58.8,9.9,171.0,,None,None,11.1
4,567552,2019-03-28T18:10:00Z,2019-03-28,R,Final,St. Louis Cardinals,Milwaukee Brewers,138,158,N,1,Jhoulys Chacin,Miles Mikolas,This is Chacin’s third career Opening Day assi...,"Mikolas, who signed a four-year extension in l...",4,5,9,Top,32,Miller Park,Milwaukee Brewers,St. Louis Cardinals,Jhoulys Chacin,Miles Mikolas,Josh Hader,2019-03-28 - St. Louis Cardinals (4) @ Milwauk...,2019-03-28T13:10:00,"43.028232,-87.970966",52.0,Partially cloudy,1.553779e+12,2019-03-28T13:10:00,35.3,None,56.22,None,50.3,50.3,0.00,None,1016.9,None,None,None,None,50.3,9.9,37.0,,21.9,None,12.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4519,633435,2021-06-30T23:10:00Z,2021-06-30,R,Completed Early: Rain,San Diego Padres,Cincinnati Reds,135,113,N,1,Vladimir Gutierrez,Joe Musgrove,,,7,5,6,Top,2602,Great American Ball Park,NaN,NaN,NaN,NaN,NaN,2021-06-30 - San Diego Padres @ Cincinnati Red...,2021-06-30T19:10:00,"39.097213,-84.506483",90.7,Overcast,1.625080e+12,2021-06-30T19:10:0

In [158]:
df_detailed = pd.merge(
    games_and_weather,
    df_filtered,
    how="outer",
    on=None,
    left_on='game_id',
    right_on='game_pk',
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_gpk", "_acw"),
    copy=True,
    indicator=False,
    validate=None,
)

In [163]:
df_detailed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 340944 entries, 0 to 340943
Columns: 145 entries, game_id to bottom
dtypes: float64(66), int64(21), object(58)
memory usage: 379.8+ MB


In [160]:
df_detailed.describe()

,cloudcover,datetime,dew,humidity,maxt,mint,precip,temp,visibility,wspd,game_date_acw,release_speed,release_pos_x,release_pos_z,batter,pitcher,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,hit_location,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score_acw,away_score_acw,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,spin_axis,delta_home_win_exp,delta_run_exp,bottom
count,340944.000000,3.409440e+05,340944.000000,340944.000000,340944.00000,340944.00000,340944.000000,340944.00000,340944.000000,340944.000000,340944.000000,339069.000000,339063.000000,339063.000000,340944.000000,340944.000000,0.0,0.0,0.0,0.0,339063.000000,295648.000000,340944.000000,340944.000000,340944.000000,339063.000000,339063.000000,339063.000000,339063.000000,33376.000000,65513.000000,103948.000000,340944.000000,340944.000000,228163.000000,228163.000000,0.0,0.0,340663.000000,0.0,0.0,339063.000000,339063.000000,339063.000000,339063.000000,339063.000000,339063.000000,339063.000000,339063.000000,216695.000000,225902.000000,225902.000000,339692.000000,335247.000000,338502.000000,340944.000000,340944.000000,340663.000000,340663.000000,340663.000000,340663.000000,340663.000000,340663.000000,340663.000000,340663.000000,339063.000000,225846.000000,225846.000000,340944.000000,340944.000000,340944.000000,340944.000000,225846.000000,340944.000000,340944.000000,340944.000000,340944.000000,340944.000000,340944.000000,340944.000000,340944.000000,340944.000000,340944.000000,338577.000000,340943.000000,340940.000000,340944.000000
mean,42.405628,1.584408e+12,55.069657,56.265656,73.56154,73.56154,0.005948,73.56154,9.593495,9.110969,737500.884788,88.781414,-0.752882,5.865934,581158.523714,582541.252390,NaN,NaN,NaN,NaN,8.038102,5.046998,1.337164,1.334310,2019.715824,-0.148204,0.637984,0.019111,2.278505,582840.631711,582453.935509,580553.032651,0.974656,4.962525,126.125448,120.897260,NaN,NaN,559640.450181,NaN,NaN,2.261558,-129.116054,-4.074383,-2.523650,25.996620,-23.627311,3.378947,1.587775,168.852475,88.317571,12.287744,88.338551,2246.482683,6.145600,596498.252490,582541.252390,559640.450181,561803.457461,590492.716805,582687.649031,599477.266037,589290.456571,597586.135395,585200.960838,54.354063,0.328598,0.374584,0.329776,0.996260,0.186365,0.156882,3.192144,38.898611,3.938377,2.329362,2.440512,2.372395,2.397479,2.500909,2.391434,2.494864,2.397479,178.192901,0.000516,0.011272,0.490371
std,31.019986,2.613638e+10,13.219358,18.566324,12.31566,12.31566,0.047400,12.31566,1.351935,4.551284,302.500044,5.884390,1.919906,0.521977,66728.282643,67277.108621,NaN,NaN,NaN,NaN,4.067805,2.635348,1.098835,0.796392,0.851337,0.870869,0.727811,0.691654,0.773107,66001.889400,66270.939997,67072.452586,0.817589,2.626482,41.054973,43.615792,NaN,NaN,68889.756338,NaN,NaN,5.799730,8.529756,2.798440,10.520155,3.844984,8.716318,0.159675,0.090134,138.298097,14.332630,28.050456,8.051723,343.939131,0.501769,33198.290583,67277.108621,68889.756338,72115.943655,61796.094860,62545.232908,51246.165009,71038.020846,57326.258091,61008.236826,0.500558,0.293901,0.397433,0.524577,0.061038,0.389401,0.591377,1.274346,22.886431,1.894137,2.677326,2.763349,2.677422,2.764316,2.786314,2.702656,2.725381,2.764316,69.433317,0.055508,0.478052,0.499908
min,0.000000,1.553107e+12,3.100000,2.750000,30.60000,30.60000,0.000000,30.60000,0.400000,0.000000,737138.000000,38.100000,-5.380000,0.800000,282332.000000,282332.000000,NaN,NaN,NaN,NaN,1.000000,1.000000,0.000

In [162]:
df_detailed[df_detailed['temp'] > 114]['venue_name'].value_counts()

Chase Field    77
Name: venue_name, dtype: int64

In [164]:
df_detailed[df_detailed['at_bat_number'] == 148 ]

,game_id,game_datetime,game_date_gpk,game_type_gpk,status,away_name,home_name,away_id,home_id,doubleheader,game_num,home_probable_pitcher,away_probable_pitcher,home_pitcher_note,away_pitcher_note,away_score_gpk,home_score_gpk,current_inning,inning_state,venue_id,venue_name,winning_team,losing_team,winning_pitcher,losing_pitcher,save_pitcher,summary,local_datetime,coordinates,cloudcover,conditions,datetime,datetimeStr,dew,heatindex,humidity,info,maxt,mint,precip,precipcover,sealevelpressure,snow,snowdepth,solarenergy,solarradiation,temp,visibility,wdir,weathertype,wgust,windchill,wspd,pitch_type,game_date_acw,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type_acw,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score_acw,away_score_acw,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bottom
180033,564909,2019-09-25T01:40:00Z,2019-09-24,R,Final,St. Louis Cardinals,Arizona Diamondbacks,138,109,N,1,Mike Leake,Jack Flaherty,Leake is 3-3 with a 4.67 ERA in nine starts si...,Flaherty stifled the Cubs over eight innings T...,2,3,19,Bottom,15,Chase Field,Arizona Diamondbacks,St. Louis Cardinals,Kevin Ginkel,John Brebbia,None,2019-09-24 - St. Louis Cardinals (2) @ Arizona...,2019-09-24T18:40:00,"33.44542,-112.066793",31.3,Partially cloudy,1.569350e+12,2019-09-24T18:40:00,57.5,82.6,41.94,None,83.0,83.0,0.0,None,1007.8,None,None,None,None,83.0,9.9,120.0,,None,None,8.8,FF,737326,93.3,-2.26,5.76,"Brebbia, John",545121,605154,single,hit_into_play,NaN,NaN,NaN,NaN,11.0,Ildemaro Vargas singles on a ground ball to le...,R,L,R,ARI,STL,X,7.0,ground_ball,0,0,2019,-1.01,1.54,-1.19,3.22,608348.0,605113.0,572233.0,2,19,75.3,101.0,NaN,NaN,425877.0,NaN,NaN,4.97793,-135.730352,-4.15101,-13.560757,29.337446,-12.462288,3.14,1.3,131.0,96.3,6.0,93.4,2453.0,6.3,564909,605154,425877.0,502671.0,669242.0,572761.0,657557.0,542303.0,664056.0,451594.0,54.17,0.567,0.519,0.9,1.0,1.0,0.0,4.0,148,1,4-Seam Fastball,2,2,2,2,2,3,3,2,Standard,Standard,213.0,0.342,1.544,1


In [167]:
df_detailed[df_detailed['delta_home_win_exp'] > .9]

,game_id,game_datetime,game_date_gpk,game_type_gpk,status,away_name,home_name,away_id,home_id,doubleheader,game_num,home_probable_pitcher,away_probable_pitcher,home_pitcher_note,away_pitcher_note,away_score_gpk,home_score_gpk,current_inning,inning_state,venue_id,venue_name,winning_team,losing_team,winning_pitcher,losing_pitcher,save_pitcher,summary,local_datetime,coordinates,cloudcover,conditions,datetime,datetimeStr,dew,heatindex,humidity,info,maxt,mint,precip,precipcover,sealevelpressure,snow,snowdepth,solarenergy,solarradiation,temp,visibility,wdir,weathertype,wgust,windchill,wspd,pitch_type,game_date_acw,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type_acw,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score_acw,away_score_acw,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bottom
22479,565420,2019-04-20T00:40:00Z,2019-04-19,R,Final,Philadelphia Phillies,Colorado Rockies,143,115,N,1,German Marquez,Vince Velasquez,Using pinpoint control with top-level velocity...,Velasquez had a no-hitter through five innings...,3,4,12,Bottom,19,Coors Field,Colorado Rockies,Philadelphia Phillies,Chad Bettis,Juan Nicasio,None,2019-04-19 - Philadelphia Phillies (3) @ Color...,2019-04-19T18:40:00,"39.756175,-104.99413",27.5,Partially cloudy,1.555699e+12,2019-04-19T18:40:00,32.6,None,28.03,None,66.7,66.7,0.0,None,1013.8,None,None,None,None,66.7,9.9,124.0,,None,None,8.8,FF,737168,93.8,-2.16,5.83,"Nicasio, Juan",453568,504379,home_run,hit_into_play,NaN,NaN,NaN,NaN,4.0,Charlie Blackmon homers (1) on a fly ball to c...,R,L,R,COL,PHI,X,NaN,fly_ball,1,2,2019,-0.86,1.24,-0.52,2.56,NaN,NaN,547172.0,2,12,164.14,14.12,NaN,NaN,592663.0,NaN,NaN,6.019232,-136.352980,-5.393502,-11.709807,26.735980,-16.060928,3.27,1.59,444.0,104.9,25.0,93.2,2168.0,5.7,565420,504379,592663.0,656555.0,514917.0,596748.0,594838.0,571437.0,596451.0,547180.0,54.76,0.980,1.875,2.0,1.0,0.0,3.0,6.0,104,4,4-Seam Fastball,2,3,2,3,3,4,4,3,Infield shift,Standard,215.0,0.904,1.953,1
134773,564990,2019-08-11T17:05:00Z,2019-08-11,R,Final,Houston Astros,Baltimore Orioles,117,110,N,1,Asher Wojciechowski,Justin Verlander,The Orioles juggled their rotation a bit to ke...,Verlander has struck out at least 10 batters i...,7,8,9,Bottom,2,Oriole Park at Camden Yards,Baltimore Orioles,Houston Astros,Richard Bleier,Roberto Osuna,None,2019-08-11 - Houston Astros (7) @ Baltimore Or...,2019-08-11T13:05:00,"39.283944,-76.621572",41.8,Partially cloudy,1.565529e+12,2019-08-11T13:05:00,54.8,83.1,36.36,None,84.4,84.4,0.0,None,1017.3,None,None,None,None,84.4,9.9,265.0,,None,None,8.4,CH,737282,85.1,-0.71,6.02,"Osuna, Roberto",547004,532077,home_run,hit_into_play,NaN,NaN,NaN,NaN,8.0,Rio Ruiz homers (7) on a fly ball to right fie...,R,L,R,BAL,HOU,X,NaN,fly_ball,2,2,2019,-0.98,0.93,0.15,1.77,NaN,NaN,642082.0,2,9,237.80,60.34,NaN,NaN,455139.0,NaN,NaN,4.006227,-123.922235,-5.652770,-10.960993,21.755618,-21.643184,3.49,1.61,417.0,103.3,29.0,86.0,1754.0,6.5,564990,532077,455139.0,649557.0,514888.0,608324.0,621043.0,502210.0,545350.0,543807.0,53.97,0.893,1.691,2.0,1.0,0.0,3.0,6.0,88,7,Changeup,6,7,6,7,7,8,8,7,Infield shift,Standard,226.0,0.904,1.923,1
185557,566070,2019-09-29T19:10:00Z,2019-09-29

In [178]:
df_detailed['covid_doubleheader'] = df_detailed.apply(lambda row: 1 if row['game_year'] in [2020, 2021] and row['doubleheader'] =='Y' else 0, axis=1)

In [183]:
american_league_teams = ['Boston Red Sox', 'Houston Astros', 'Chicago White Sox', 'Tampa Bay Rays', 'Oakland Athletics', 'Seattle Mariners', 'New York Yankees', 'Toronto Blue Jays', 'Los Angeles Angels', 'Cleveland Indians', 'Detroit Tigers', 'Kansas City Royals', 'Minnesota Twins', 'Texas Rangers', 'Baltimore Orioles']

In [196]:
df_detailed['designated_hitter'] = df_detailed.apply(lambda row: 1 if row['home_name'] in american_league_teams or row['game_year'] == 2020 else 0, axis = 1)

In [205]:
df_detailed[(df_detailed['home_name'] == 'New York Mets') & (df_detailed['designated_hitter'] == 1)]

,game_id,game_datetime,game_date_gpk,game_type_gpk,status,away_name,home_name,away_id,home_id,doubleheader,game_num,home_probable_pitcher,away_probable_pitcher,home_pitcher_note,away_pitcher_note,away_score_gpk,home_score_gpk,current_inning,inning_state,venue_id,venue_name,winning_team,losing_team,winning_pitcher,losing_pitcher,save_pitcher,summary,local_datetime,coordinates,cloudcover,conditions,datetime,datetimeStr,dew,heatindex,humidity,info,maxt,mint,precip,precipcover,sealevelpressure,snow,snowdepth,solarenergy,solarradiation,temp,visibility,wdir,weathertype,wgust,windchill,wspd,pitch_type,game_date_acw,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type_acw,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score_acw,away_score_acw,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bottom,covid_doubleheader,designated_hitter
185884,631242,2020-07-24T20:10:00Z,2020-07-24,R,Final,Atlanta Braves,New York Mets,144,121,N,1,Jacob deGrom,Mike Soroka,The two-time reigning National League Cy Young...,Soroka will become the youngest Opening Day st...,0,1,9,Top,3289,Citi Field,New York Mets,Atlanta Braves,Seth Lugo,Chris Martin,Edwin Diaz,2020-07-24 - Atlanta Braves (0) @ New York Met...,2020-07-24T16:10:00,"40.757134,-73.84584",70.0,Partially cloudy,1.595607e+12,2020-07-24T16:10:00,68.6,None,73.04,None,77.9,77.9,0.0,None,1018.1,None,None,None,None,77.9,9.9,39.0,,None,None,5.8,FF,737630,99.0,-1.16,5.52,"deGrom, Jacob",660670,594798,strikeout,swinging_strike,NaN,NaN,NaN,NaN,5.0,Ronald Acuna strikes out swinging.,R,R,R,NYM,ATL,S,2.0,NaN,2,2,2020,-0.42,1.32,-0.04,2.77,NaN,NaN,NaN,0,1,NaN,NaN,NaN,NaN,467092.0,NaN,NaN,4.111002,-144.174062,-5.285107,-6.853423,31.668520,-12.679647,3.52,1.67,NaN,NaN,NaN,100.8,2315.0,7.1,631242,594798,467092.0,624413.0,429664.0,643446.0,642708.0,605204.0,607043.0,624424.0,53.37,NaN,NaN,0.00,1.0,0.0,0.0,NaN,1,6,4-Seam Fastball,0,0,0,0,0,0,0,0,Standard,Standard,217.0,0.022,-0.215,0,0,1
185885,631242,2020-07-24T20:10:00Z,2020-07-24,R,Final,Atlanta Braves,New York Mets,144,121,N,1,Jacob deGrom,Mike Soroka,The two-time reigning National League Cy Young...,Soroka will become the youngest Opening Day st...,0,1,9,Top,3289,Citi Field,New York Mets,Atlanta Braves,Seth Lugo,Chris Martin,Edwin Diaz,2020-07-24 - Atlanta Braves (0) @ New York Met...,2020-07-24T16:10:00,"40.757134,-73.84584",70.0,Partially cloudy,1.595607e+12,2020-07-24T16:10:00,68.6,None,73.04,None,77.9,77.9,0.0,None,1018.1,None,None,None,None,77.9,9.9,39.0,,None,None,5.8,FF,737630,98.5,-0.99,5.53,"deGrom, Jacob",645277,594798,field_out,hit_into_play,NaN,NaN,NaN,NaN,2.0,"Ozzie Albies grounds out, third baseman Jeff M...",R,L,R,NYM,ATL,X,5.0,ground_ball,0,2,2020,-0.69,1.40,-0.26,2.78,NaN,NaN,NaN,1,1,167.31,145.07,NaN,NaN,467092.0,NaN,NaN,3.658128,-143.425662,-5.368227,-10.377674,32.528566,-11.795985,3.28,1.56,15.0,94.4,-10.0,100.0,2383.0,7.0,631242,594798,467092.0,624413.0,429664.0,643446.0,642708.0,605204.0,607043.0,624424.0,53.49,0.173,0.163,0.00,1.0,0.0,0.0,2.0,2,4,4-Seam Fastball,0,0,0,0,0,0,0,0,Infield shift,Standard,212.0,0.015,-0.103,0,0,1
185886,631242,2020-07-24T20:10:00Z,2020-07-24,R,Final,Atlanta Braves,New York Mets,144,121,N,1,Ja

In [206]:
df_detailed[(df_detailed['home_name'] == 'Oakland Athletics') & (df_detailed['designated_hitter'] == 0)]

,game_id,game_datetime,game_date_gpk,game_type_gpk,status,away_name,home_name,away_id,home_id,doubleheader,game_num,home_probable_pitcher,away_probable_pitcher,home_pitcher_note,away_pitcher_note,away_score_gpk,home_score_gpk,current_inning,inning_state,venue_id,venue_name,winning_team,losing_team,winning_pitcher,losing_pitcher,save_pitcher,summary,local_datetime,coordinates,cloudcover,conditions,datetime,datetimeStr,dew,heatindex,humidity,info,maxt,mint,precip,precipcover,sealevelpressure,snow,snowdepth,solarenergy,solarradiation,temp,visibility,wdir,weathertype,wgust,windchill,wspd,pitch_type,game_date_acw,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type_acw,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score_acw,away_score_acw,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bottom,covid_doubleheader,designated_hitter


In [ ]:
df_detailed.apply(lambda row: , axis=1)

next step: create a function which can take in a player (name or player_id?), type of stats requested (pitcher or hitter), a start date, and an end date, and can return cumulated stats through that time period

In [236]:
# probably best off grabbing all the data for each player and trimming it myself - speed up, cut down on api calls

# instead of career stats we'll go off last two years to bias towards recency
def derived_cumulative_stats(player, stats_type, start_date='2017-01-01', end_date=datetime.now()):
    if type(player) == str:
        player = player.split(', ')
        try:
            player = playerid_lookup(player[0], player[1])
            if len(player) >1:
                raise NameError(player)
        except:
            raise NameError(player)
    if stats_type = 'pitcher':
        history = statcast_pitcher(start_date, end_date, player).sort_values(['game_date', 'game_pk','home_team', 'inning', 'outs_when_up', 'at_bat_number', 'pitch_number']).reset_index(drop=True)
        history = history[history['game_type'] == 'R']
    #     derive hits per inning, batters faced per appearence,
    #     derive R/L splits for all the above
    #     pitch specific stats - assemble pitch types in df with how often thrown, avg pfx_x, pfx_z, vx0, vy0, vz0, ax, ay, az, release_spin_rate, release_extension

    #   PITCH METRICS
        pitch_list = [x for x in history['pitch_type'].value_counts().index]
        pitches = pd.DataFrame()
        for each in pitch_list:
            metrics = history[history['pitch_type'] == each].describe()
            pitches[each+'_quantity'] =  metrics.loc['count', 'batter']
            pitches[each+'_avg_release_speed'] = metrics.loc['mean', 'release_speed']
            pitches[each+'_avg_pfx_x'] = metrics.loc['mean', 'pfx_x']
            pitches[each+'_avg_pfx_z'] = metrics.loc['mean', 'pfx_z']
            pitches[each+'_avg_vx0'] = metrics.loc['mean', 'vx0']
            pitches[each+'_avg_vy0'] = metrics.loc['mean', 'vy0']
            pitches[each+'_avg_vz0'] = metrics.loc['mean', 'vz0']
            pitches[each+'_avg_ax'] = metrics.loc['mean', 'ax']
            pitches[each+'_avg_ay'] = metrics.loc['mean', 'ay']
            pitches[each+'_avg_az'] = metrics.loc['mean', 'az']
            pitches[each+'_avg_release_spin_rate'] = metrics.loc['mean', 'release_spin_rate']
            pitches[each+'_avg_release_extension'] = metrics.loc['mean', 'release_extension']

    #     filter history down to PA-enders
        history = history[history['events'].isin(pa_ending_events)]

    #    PLAYER METRICS
        games = history['game_pk'].unique()
        at_bat_list = []
#         inning_list = []
        num_hits_list = []
        for game in games:
            inning = history[history['game_pk'] == game].tail(1)['inning']
#             inning_list.append(inning)
            for i in range(1, inning+1):
                num_hits = len(history[(history['game_pk'] == game) & (history['inning'] == i) & (history['event'].isin(hit_events))])
                num_hits_list.append(num_hits)
            at_bats = len(history[history['game_pk'] == game])
            at_bat_list.append(at_bats)
        pitches['avg_PAs_per_apparence'] = np.array(at_bat_list).mean()
        pitches['avg_hits_per_inning'] = np.array(num_hits_list).mean()
        
#         L/R splits
        l_pas = len(history[history['stand'] == 'L'])
        r_pas = len(history[history['stand'] == 'R'])
        l_hits = len(history[(history['stand'] == 'L') & (history['event'].isin(hit_events))])
        r_hits = len(history[(history['stand'] == 'R') & (history['event'].isin(hit_events))])
        pitches['H/PA'] = (l_hits + r_hits)/(l_pas + r_pas)
        pitches['against_lefties_H/PA'] = (l_hits)/(l_pas)
        pitches['against_righties_H/PA'] = (r_hits)/(r_pas)

        return(pitches)
    elif stats_type = 'batter':
        history = statcast_batter(start_date, end_date, player).sort_values(['game_date', 'game_pk','home_team', 'inning', 'outs_when_up', 'at_bat_number', 'pitch_number']).reset_index(drop=True)
        history = history[history['game_type'] == 'R']

#     filter history down to PA-enders
        history = history[history['events'].isin(pa_ending_events)]
    
#     H/PA per pitch type
#     K/PA per pitch type
#     H/PA vs righties, lefties
#     PA/G
#     avg_launch_angle
#     avg_launch_speed
#     xBA based on estimated_ba_using_speedangle



        pitches = history['pitch_type'].unique()
        games = history['game_pk'].unique()
        pas_list = []
        
        hits = pd.DataFrame()
        
        for game in games:
            pas = len[history['game_pk'] == game]
            pas_list.append(pas)
        
        hits['PA/G'] = np.array(pas_list).mean()
        
        for pitch in pitches:
            pas = len[history['pitch_type'] == pitch]
            hits = len(history[(history['pitch_type'] == pitch) & history['event'].isin(hit_events)])
            strikeouts = len(history[(history['pitch_type'] == pitch)& (history['event'].isin(['strikeout', 'strikeout_double_play']))])
            
            hits[pitch+'_H/PA'] = hits/pas
            hits[pitch+'_K/PA'] = strikeouts/pas
#         righty/lefty split
        pas = 


# senario_df = senario_df.sort_values('hit').drop_duplicates(subset=['date', 'number', 'player'], keep='last')


In [237]:
# playerid_lookup('deGrom, Jacob'.split(', '))

# type('degrom, jacob') == str
derived_cumulative_stats('degrom, jacob', None, None)

594798

In [224]:
derived_cumulative_stats('deGrom, Jacob', None, None)

<class 'str'>


In [244]:
jdg = statcast_pitcher('2019-01-01', '2021-07-28', 594798).sort_values(['game_date', 'game_pk','home_team', 'inning', 'outs_when_up', 'at_bat_number', 'pitch_number']).reset_index(drop=True)

Gathering Player Data


In [284]:
pete = statcast_batter('2019-01-01', '2021-07-28', 624413).sort_values(['game_date', 'game_pk','home_team', 'inning', 'outs_when_up', 'at_bat_number', 'pitch_number']).reset_index(drop=True)

Gathering Player Data


In [287]:
pete

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,SL,2019-03-28,85.6,-3.47,5.22,"Alonso, Pete",624413,453286,NaN,swinging_strike,NaN,NaN,NaN,NaN,6.0,Pete Alonso strikes out on a foul tip.,R,R,R,WSH,NYM,S,NaN,NaN,0,0,2019,0.29,0.02,0.55,2.31,NaN,NaN,NaN,1,1,Top,NaN,NaN,NaN,NaN,543228,NaN,NaN,8.858654,-124.347396,-0.623432,1.238465,23.949670,-32.052933,3.48,1.60,NaN,NaN,NaN,85.2,2370.0,6.1,565895,453286,543228,475582,572821,543685,607208,665742,645302,594809,54.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,Slider,0,0,0,0,0,0,0,0,Infield shift,Standard,95.0,0.000,-0.030
1,FF,2019-03-28,95.5,-3.37,5.10,"Alonso, Pete",624413,453286,NaN,ball,NaN,NaN,NaN,NaN,12.0,Pete Alonso strikes out on a foul tip.,R,R,R,WSH,NYM,B,NaN,NaN,0,1,2019,-1.13,1.16,1.73,2.60,NaN,NaN,NaN,1,1,Top,NaN,NaN,NaN,NaN,543228,NaN,NaN,15.799330,-138.114546,-3.468802,-17.704078,28.283129,-16.918344,3.29,1.59,NaN,NaN,NaN,94.9,2632.0,6.1,565895,453286,543228,475582,572821,543685,607208,665742,645302,594809,54.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,4-Seam Fastball,0,0,0,0,0,0,0,0,Infield shift,Standard,224.0,0.000,0.022
2,FF,2019-03-28,96.2,-3.47,5.12,"Alonso, Pete",624413,453286,NaN,foul,NaN,NaN,NaN,NaN,6.0,Pete Alonso strikes out on a foul tip.,R,R,R,WSH,NYM,S,NaN,NaN,1,1,2019,-1.14,1.25,0.44,2.51,NaN,NaN,NaN,1,1,Top,NaN,NaN,NaN,NaN,543228,NaN,NaN,12.876967,-139.467553,-4.132844,-17.599947,29.346292,-15.210372,3.48,1.60,178.0,80.8,20.0,96.1,2702.0,6.3,565895,453286,543228,475582,572821,543685,607208,665742,645302,594809,54.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3,4-Seam Fastball,0,0,0,0,0,0,0,0,Infield shift,Standard,222.0,0.000,-0.044
3,FC,2019-03-28,87.0,-3.47,5.12,"Alonso, Pete",624413,453286,NaN,foul,NaN,NaN,NaN,NaN,8.0,Pete Alonso strikes out on a foul tip.,R,R,R,WSH,NYM,S,NaN,NaN,1,2,2019,0.08,0.31,0.01,1.91,NaN,NaN,NaN,1,1,Top,NaN,NaN,NaN,NaN,543228,NaN,NaN,8.090192,-126.492662,-2.173833,-0.633117,23.666096,-28.671571,3.48,1.60,NaN,NaN,NaN,86.3,2327.0,5.7,565895,453286,543228,475582,572821,543685,607208,665742,645302,594809,54.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,4,Cutter,0,0,0,0,0,0,0,0,Infield shift,Standard,165.0,0.000,0.000
4,FF,2019-03-28,97.5,-3.33,5.09,"Alonso, Pete",624413,453286,NaN,ball,NaN,NaN,NaN,NaN,14.0,Pete Alonso strikes out on a foul tip.,R,R,R,WSH,NYM,B,NaN,NaN,1,2,2019,-0.84,1.40,1.21,1.98,NaN,NaN,NaN,1,1,Top,NaN,NaN,NaN,NaN,543228,NaN,NaN,14.057026,-141.217661,-5.993151,-14.301712,30.783424,-12.334048,3.19,1.56,NaN,NaN,NaN,97.2,2545.0,6.3,565895,453286,543228,475582,572821,543685,607208,665742,645302,594809,54.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,5,4-Seam Fastball,0,0,0,0,0,0,0,0,Infield shift,Standard,211.0,0.000,0.031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4989,SI,2021-0

In [288]:
pete.iso_value.value_counts()

0.0    1140
3.0      91
1.0      49
2.0       2
Name: iso_value, dtype: int64